In [1]:
import pandas as pd
import re
import pymysql
from sqlalchemy import create_engine
import getpass

In [2]:
total_accidents_2018 = pd.read_excel('./files_for_lab/2018_Accidentalidad.xlsx')
total_accidents_2018

,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,...,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO ACCIDENTE,Tipo Vehiculo,TIPO PERSONA,SEXO,LESIVIDAD,Tramo Edad
0,2018-01-01,DE 00:00 A 00:59,LUNES,USERA,CALLE DE SAN BASILIO - CALLE DEL CRISTO DE LA ...,0,2018/1,NO,NO,NO,...,NO,NO,SI,1,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HG,DE 15 A 17 AÑOS
1,2018-01-01,DE 00:00 A 00:59,LUNES,USERA,CALLE DE SAN BASILIO - CALLE DEL CRISTO DE LA ...,0,2018/1,NO,NO,NO,...,NO,NO,SI,1,ATROPELLO,NO ASIGNADO,TESTIGO,HOMBRE,IL,DE 30 A 34 ANOS
2,2018-01-01,DE 00:00 A 00:59,LUNES,USERA,CALLE DE SAN BASILIO - CALLE DEL CRISTO DE LA ...,0,2018/1,NO,NO,NO,...,NO,NO,SI,1,ATROPELLO,TURISMO,CONDUCTOR,HOMBRE,IL,DE 35 A 39 AÑOS
3,2018-01-01,DE 1:00 A 1:59,LUNES,HORTALEZA,AVENIDA DE FRANCISCO PI Y MARGALL - AVENIDA DE...,,2018/3,NO,NO,NO,...,NO,NO,SI,1,CHOQUE CON OBJETO FIJO,NO ASIGNADO,TESTIGO,HOMBRE,IL,DE 21 A 24 AÑOS
4,2018-01-01,DE 1:00 A 1:59,LUNES,HORTALEZA,AVENIDA DE FRANCISCO PI Y MARGALL - AVENIDA DE...,,2018/3,NO,NO,NO,...,NO,NO,SI,1,CHOQUE CON OBJETO FIJO,NO ASIGNADO,TESTIGO,MUJER,IL,DE 40 A 44 AÑOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30117,2018-12-31,DE 21:00 A 21:59,LUNES,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA ...,0,2019/18,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,TURISMO,VIAJERO,HOMBRE,IL,DE 30 A 34 ANOS
30118,2018-12-31,DE 21:00 A 21:59,LUNES,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA ...,0,2019/18,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,AUTO-TAXI,CONDUCTOR,HOMBRE,IL,DE 40 A 44 AÑOS
30119,2018-12-31,DE 23:00 A 23:59,LUNES,HORTALEZA,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,0,2019/53,NO,NO,NO,...,NO,NO,SI,1,OTRAS CAUSAS,TURISMO,CONDUCTOR,MUJER,IL,DE 21 A 24 AÑOS
30120,2018-12-31,DE 23:00 A 23:59,LUNES,HORTALEZA,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,0,2019/53,NO,NO,NO,...,NO,NO,SI,1,OTRAS CAUSAS,TURISMO,CONDUCTOR,HOMBRE,HL,DE 50 A 54 AÑOS


In [3]:
total_accidents_2018['TIPO PERSONA'].value_counts()

CONDUCTOR                                   19333
VIAJERO                                      5365
TESTIGO                                      3659
PEATON                                       1765
Name: TIPO PERSONA, dtype: int64

In [4]:
total_accidents_2018['LESIVIDAD'].value_counts()

IL                                          15574
HL                                          12497
NO ASIGNADA                                  1118
HG                                            896
MT                                             37
Name: LESIVIDAD, dtype: int64

In [5]:
total_accidents_2018.columns

Index(['FECHA', 'RANGO HORARIO', 'DIA SEMANA', 'DISTRITO', 'LUGAR ACCIDENTE',
       'Nº', 'Nº PARTE', 'CPFA Granizo', 'CPFA Hielo', 'CPFA Lluvia',
       'CPFA Niebla', 'CPFA Seco', 'CPFA Nieve', 'CPSV Mojada', 'CPSV Aceite',
       'CPSV Barro', 'CPSV Grava Suelta', 'CPSV Hielo', 'CPSV Seca Y Limpia',
       'Nº VICTIMAS', 'TIPO ACCIDENTE', 'Tipo Vehiculo', 'TIPO PERSONA',
       'SEXO', 'LESIVIDAD', 'Tramo Edad'],
      dtype='object')

In [6]:
total_accidents_2018.isna().sum()

FECHA                 0
RANGO HORARIO         0
DIA SEMANA            0
DISTRITO              0
LUGAR ACCIDENTE       0
Nº                    0
Nº PARTE              0
CPFA Granizo          0
CPFA Hielo            0
CPFA Lluvia           0
CPFA Niebla           0
CPFA Seco             0
CPFA Nieve            0
CPSV Mojada           0
CPSV Aceite           0
CPSV Barro            0
CPSV Grava Suelta     0
CPSV Hielo            0
CPSV Seca Y Limpia    0
Nº VICTIMAS           0
TIPO ACCIDENTE        0
Tipo Vehiculo         0
TIPO PERSONA          0
SEXO                  0
LESIVIDAD             0
Tramo Edad            0
dtype: int64

In [7]:
total_accidents_2018['DISTRITO'].unique()

array(['USERA                         ', 'HORTALEZA                     ',
       'SAN BLAS                      ', 'CIUDAD LINEAL                 ',
       'FUENCARRAL-EL PARDO           ', 'MORATALAZ                     ',
       'ARGANZUELA                    ', 'CENTRO                        ',
       'SALAMANCA                     ', 'CHAMARTIN                     ',
       'CARABANCHEL                   ', 'RETIRO                        ',
       'TETUAN                        ', 'MONCLOA-ARAVACA               ',
       'CHAMBERI                      ', 'VICALVARO                     ',
       'VILLAVERDE                    ', 'VILLA DE VALLECAS             ',
       'PUENTE DE VALLECAS            ', 'LATINA                        ',
       'BARAJAS                       '], dtype=object)

In [8]:
for col in total_accidents_2018.columns:
    print(total_accidents_2018[col].unique())

['2018-01-01T00:00:00.000000000' '2018-01-02T00:00:00.000000000'
 '2018-01-03T00:00:00.000000000' '2018-01-04T00:00:00.000000000'
 '2018-01-05T00:00:00.000000000' '2018-01-06T00:00:00.000000000'
 '2018-01-07T00:00:00.000000000' '2018-01-08T00:00:00.000000000'
 '2018-01-09T00:00:00.000000000' '2018-01-10T00:00:00.000000000'
 '2018-01-11T00:00:00.000000000' '2018-01-12T00:00:00.000000000'
 '2018-01-13T00:00:00.000000000' '2018-01-14T00:00:00.000000000'
 '2018-01-15T00:00:00.000000000' '2018-01-16T00:00:00.000000000'
 '2018-01-17T00:00:00.000000000' '2018-01-18T00:00:00.000000000'
 '2018-01-19T00:00:00.000000000' '2018-01-20T00:00:00.000000000'
 '2018-01-21T00:00:00.000000000' '2018-01-22T00:00:00.000000000'
 '2018-01-23T00:00:00.000000000' '2018-01-24T00:00:00.000000000'
 '2018-01-25T00:00:00.000000000' '2018-01-26T00:00:00.000000000'
 '2018-01-27T00:00:00.000000000' '2018-01-28T00:00:00.000000000'
 '2018-01-29T00:00:00.000000000' '2018-01-30T00:00:00.000000000'
 '2018-01-31T00:00:00.000

In [9]:
total_accidents_2017 = pd.read_excel('./files_for_lab/2017_Accidentalidad.xlsx')
total_accidents_2017

,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,...,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO ACCIDENTE,Tipo Vehiculo,TIPO PERSONA,SEXO,LESIVIDAD,Tramo Edad
0,2017-01-01,DE 1:00 A 1:59,DOMINGO,VILLA DE VALLECAS,AVENIDA DEL MAYORAZGO NUM ...,26,2017/76,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,FURGONETA,CONDUCTOR,HOMBRE,IL,DE 25 A 29 AÑOS
1,2017-01-01,DE 1:00 A 1:59,DOMINGO,VILLA DE VALLECAS,AVENIDA DEL MAYORAZGO NUM ...,26,2017/76,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,FURGONETA,CONDUCTOR,MUJER,HL,DE 30 A 34 ANOS
2,2017-01-01,DE 2:00 A 2:59,DOMINGO,VILLAVERDE,CALLE DE SANTA JOAQUINA DE VEDRUNA NUM ...,3,2017/1,NO,NO,NO,...,NO,NO,SI,1,CHOQUE CON OBJETO FIJO,TURISMO,CONDUCTOR,HOMBRE,HG,DE 21 A 24 AÑOS
3,2017-01-01,DE 5:00 A 5:59,DOMINGO,ARGANZUELA,CALLE 30 15RA KM. ...,30,2017/30,NO,NO,NO,...,NO,NO,SI,1,CHOQUE CON OBJETO FIJO,TURISMO,CONDUCTOR,MUJER,HL,DE 45 A 49 AÑOS
4,2017-01-01,DE 5:00 A 5:59,DOMINGO,ARGANZUELA,CALLE 30 15RA KM. ...,30,2017/30,NO,NO,NO,...,NO,NO,SI,1,CHOQUE CON OBJETO FIJO,TURISMO,VIAJERO,MUJER,IL,DE 10 A 14 AÑOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29790,2017-12-31,DE 18:00 A 18:59,DOMINGO,TETUAN,PASEO DE LA CASTELLANA - PLAZA DE LIMA ...,0,2018/307,NO,NO,SI,...,NO,NO,NO,1,ATROPELLO,TURISMO,CONDUCTOR,HOMBRE,IL,DE 65 A 69 AÑOS
29791,2017-12-31,DE 18:00 A 18:59,DOMINGO,TETUAN,PASEO DE LA CASTELLANA - PLAZA DE LIMA ...,0,2018/307,NO,NO,SI,...,NO,NO,NO,1,ATROPELLO,AUTOBUS-AUTOCAR,CONDUCTOR,HOMBRE,IL,DE 35 A 39 AÑOS
29792,2017-12-31,DE 21:00 A 21:59,DOMINGO,CHAMBERI,PASEO DE EDUARDO DATO NUM ...,2,2018/19,NO,NO,SI,...,NO,NO,NO,1,COLISIÓN MÚLTIPLE,TURISMO,CONDUCTOR,HOMBRE,IL,DE 45 A 49 AÑOS
29793,2017-12-31,DE 21:00 A 21:59,DOMINGO,CHAMBERI,PASEO DE EDUARDO DATO NUM ...,2,2018/19,NO,NO,SI,...,NO,NO,NO,1,COLISIÓN MÚLTIPLE,TURISMO,CONDUCTOR,MUJER,IL,DE 45 A 49 AÑOS


In [10]:
total_accidents_2016 = pd.read_excel('./files_for_lab/2016_Accidentalidad.xlsx')
total_accidents_2016

,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,...,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO ACCIDENTE,Tipo Vehiculo,TIPO PERSONA,SEXO,LESIVIDAD,Tramo Edad
0,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 6 A 9 AÑOS
1,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 40 A 44 AÑOS
2,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 35 A 39 AÑOS
3,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,PEATON,MUJER,HL,DE 35 A 39 AÑOS
4,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,TESTIGO,MUJER,IL,DE 40 A 44 AÑOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29196,2016-12-31,DE 18:00 A 18:59,SABADO,MORATALAZ,AVENIDA DEL DOCTOR GARCIA TAPIA NUM ...,232,2017/49,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,TURISMO,CONDUCTOR,NO ASIGNADO,NO ASIGNADA,DESCONOCIDA
29197,2016-12-31,DE 18:00 A 18:59,SABADO,MORATALAZ,AVENIDA DEL DOCTOR GARCIA TAPIA NUM ...,232,2017/49,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,BICICLETA,CONDUCTOR,MUJER,HL,DE 6 A 9 AÑOS
29198,2016-12-31,DE 20:00 A 20:59,SABADO,CARABANCHEL,CALLE DE EUGENIA DE MONTIJO NUM ...,83,2017/14,NO,NO,NO,...,NO,NO,SI,1,CAÍDA MOTOCICLETA,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 21 A 24 AÑOS
29199,2016-12-31,DE 23:00 A 23:59,SABADO,MONCLOA-ARAVACA,PASEO DEL PINTOR ROSALES NUM ...,44,2017/35,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,TURISMO,CONDUCTOR,HOMBRE,IL,DE 21 A 24 AÑOS


In [11]:
def clean_file(row):
    #Changing headers names
    cols1 = []
    for column in row.columns:
        new_column_name = column.replace(' ', '_')
        new_column_name = new_column_name.replace('º', 'ro')
        cols1.append(new_column_name.lower())
    row.columns = cols1
    # I'm going to clean this columns because all values have spaces at the start and at the end
    list_to_clean = ['distrito', 'lugar_accidente', 'tipo_accidente', 'tipo_vehiculo', 'tipo_persona', 'lesividad']
    for column in list_to_clean:
        for i in range(len(row['lesividad'])):
            row[column][i] = str(row[column][i])
            row[column][i] = row[column][i].strip()
    row = row.reset_index(drop = True)
    # I don't need witness("TESTIGO") because i want statistics about people who was involved into the accident
    row = row.drop(row[row['tipo_persona']=='TESTIGO'].index)
    row = row.reset_index(drop = True)
    return row

In [12]:
total_accidents_2018['DIA SEMANA'].value_counts()

VIERNES      5308
MIERCOLES    4660
MARTES       4570
JUEVES       4480
LUNES        4430
SABADO       3463
DOMINGO      3211
Name: DIA SEMANA, dtype: int64

In [13]:
total_accidents_2016['TIPO PERSONA'].value_counts()

CONDUCTOR                                   18575
VIAJERO                                      5189
TESTIGO                                      3724
PEATON                                       1713
Name: TIPO PERSONA, dtype: int64

In [14]:
clean_2016 = clean_file(total_accidents_2016)
clean_2016

<ipython-input-11-dc4b31b9055a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[column][i] = str(row[column][i])
<ipython-input-11-dc4b31b9055a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[column][i] = row[column][i].strip()


,fecha,rango_horario,dia_semana,distrito,lugar_accidente,nro,nro_parte,cpfa_granizo,cpfa_hielo,cpfa_lluvia,...,cpsv_grava_suelta,cpsv_hielo,cpsv_seca_y_limpia,nro_victimas,tipo_accidente,tipo_vehiculo,tipo_persona,sexo,lesividad,tramo_edad
0,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 6 A 9 AÑOS
1,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 40 A 44 AÑOS
2,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 35 A 39 AÑOS
3,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,PEATON,MUJER,HL,DE 35 A 39 AÑOS
4,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,TURISMO,CONDUCTOR,HOMBRE,IL,DE 30 A 34 ANOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25472,2016-12-31,DE 18:00 A 18:59,SABADO,MORATALAZ,AVENIDA DEL DOCTOR GARCIA TAPIA NUM,232,2017/49,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,TURISMO,CONDUCTOR,NO ASIGNADO,NO ASIGNADA,DESCONOCIDA
25473,2016-12-31,DE 18:00 A 18:59,SABADO,MORATALAZ,AVENIDA DEL DOCTOR GARCIA TAPIA NUM,232,2017/49,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,BICICLETA,CONDUCTOR,MUJER,HL,DE 6 A 9 AÑOS
25474,2016-12-31,DE 20:00 A 20:59,SABADO,CARABANCHEL,CALLE DE EUGENIA DE MONTIJO NUM,83,2017/14,NO,NO,NO,...,NO,NO,SI,1,CAÍDA MOTOCICLETA,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 21 A 24 AÑOS
25475,2016-12-31,DE 23:00 A 23:59,SABADO,MONCLOA-ARAVACA,PASEO DEL PINTOR ROSALES NUM,44,2017/35,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,TURISMO,CONDUCTOR,HOMBRE,IL,DE 21 A 24 AÑOS


In [15]:
total_accidents_2017['TIPO PERSONA'].value_counts()

CONDUCTOR                                   19105
VIAJERO                                      5304
TESTIGO                                      3699
PEATON                                       1687
Name: TIPO PERSONA, dtype: int64

In [16]:
clean_2017 = clean_file(total_accidents_2017)
clean_2017

<ipython-input-11-dc4b31b9055a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[column][i] = str(row[column][i])
<ipython-input-11-dc4b31b9055a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[column][i] = row[column][i].strip()


,fecha,rango_horario,dia_semana,distrito,lugar_accidente,nro,nro_parte,cpfa_granizo,cpfa_hielo,cpfa_lluvia,...,cpsv_grava_suelta,cpsv_hielo,cpsv_seca_y_limpia,nro_victimas,tipo_accidente,tipo_vehiculo,tipo_persona,sexo,lesividad,tramo_edad
0,2017-01-01,DE 1:00 A 1:59,DOMINGO,VILLA DE VALLECAS,AVENIDA DEL MAYORAZGO NUM,26,2017/76,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,FURGONETA,CONDUCTOR,HOMBRE,IL,DE 25 A 29 AÑOS
1,2017-01-01,DE 1:00 A 1:59,DOMINGO,VILLA DE VALLECAS,AVENIDA DEL MAYORAZGO NUM,26,2017/76,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,FURGONETA,CONDUCTOR,MUJER,HL,DE 30 A 34 ANOS
2,2017-01-01,DE 2:00 A 2:59,DOMINGO,VILLAVERDE,CALLE DE SANTA JOAQUINA DE VEDRUNA NUM,3,2017/1,NO,NO,NO,...,NO,NO,SI,1,CHOQUE CON OBJETO FIJO,TURISMO,CONDUCTOR,HOMBRE,HG,DE 21 A 24 AÑOS
3,2017-01-01,DE 5:00 A 5:59,DOMINGO,ARGANZUELA,CALLE 30 15RA KM.,30,2017/30,NO,NO,NO,...,NO,NO,SI,1,CHOQUE CON OBJETO FIJO,TURISMO,CONDUCTOR,MUJER,HL,DE 45 A 49 AÑOS
4,2017-01-01,DE 5:00 A 5:59,DOMINGO,ARGANZUELA,CALLE 30 15RA KM.,30,2017/30,NO,NO,NO,...,NO,NO,SI,1,CHOQUE CON OBJETO FIJO,TURISMO,VIAJERO,MUJER,IL,DE 10 A 14 AÑOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26091,2017-12-31,DE 18:00 A 18:59,DOMINGO,TETUAN,PASEO DE LA CASTELLANA - PLAZA DE LIMA,0,2018/307,NO,NO,SI,...,NO,NO,NO,1,ATROPELLO,TURISMO,CONDUCTOR,HOMBRE,IL,DE 65 A 69 AÑOS
26092,2017-12-31,DE 18:00 A 18:59,DOMINGO,TETUAN,PASEO DE LA CASTELLANA - PLAZA DE LIMA,0,2018/307,NO,NO,SI,...,NO,NO,NO,1,ATROPELLO,AUTOBUS-AUTOCAR,CONDUCTOR,HOMBRE,IL,DE 35 A 39 AÑOS
26093,2017-12-31,DE 21:00 A 21:59,DOMINGO,CHAMBERI,PASEO DE EDUARDO DATO NUM,2,2018/19,NO,NO,SI,...,NO,NO,NO,1,COLISIÓN MÚLTIPLE,TURISMO,CONDUCTOR,HOMBRE,IL,DE 45 A 49 AÑOS
26094,2017-12-31,DE 21:00 A 21:59,DOMINGO,CHAMBERI,PASEO DE EDUARDO DATO NUM,2,2018/19,NO,NO,SI,...,NO,NO,NO,1,COLISIÓN MÚLTIPLE,TURISMO,CONDUCTOR,MUJER,IL,DE 45 A 49 AÑOS


In [17]:
clean_2018 = clean_file(total_accidents_2018)
clean_2018

<ipython-input-11-dc4b31b9055a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[column][i] = str(row[column][i])
<ipython-input-11-dc4b31b9055a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[column][i] = row[column][i].strip()


,fecha,rango_horario,dia_semana,distrito,lugar_accidente,nro,nro_parte,cpfa_granizo,cpfa_hielo,cpfa_lluvia,...,cpsv_grava_suelta,cpsv_hielo,cpsv_seca_y_limpia,nro_victimas,tipo_accidente,tipo_vehiculo,tipo_persona,sexo,lesividad,tramo_edad
0,2018-01-01,DE 00:00 A 00:59,LUNES,USERA,CALLE DE SAN BASILIO - CALLE DEL CRISTO DE LA ...,0,2018/1,NO,NO,NO,...,NO,NO,SI,1,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HG,DE 15 A 17 AÑOS
1,2018-01-01,DE 00:00 A 00:59,LUNES,USERA,CALLE DE SAN BASILIO - CALLE DEL CRISTO DE LA ...,0,2018/1,NO,NO,NO,...,NO,NO,SI,1,ATROPELLO,TURISMO,CONDUCTOR,HOMBRE,IL,DE 35 A 39 AÑOS
2,2018-01-01,DE 1:00 A 1:59,LUNES,HORTALEZA,AVENIDA DE FRANCISCO PI Y MARGALL - AVENIDA DE...,,2018/3,NO,NO,NO,...,NO,NO,SI,1,CHOQUE CON OBJETO FIJO,TURISMO,CONDUCTOR,HOMBRE,IL,DE 21 A 24 AÑOS
3,2018-01-01,DE 1:00 A 1:59,LUNES,HORTALEZA,AVENIDA DE FRANCISCO PI Y MARGALL - AVENIDA DE...,,2018/3,NO,NO,NO,...,NO,NO,SI,1,CHOQUE CON OBJETO FIJO,TURISMO,VIAJERO,HOMBRE,HG,DE 21 A 24 AÑOS
4,2018-01-01,DE 1:00 A 1:59,LUNES,SAN BLAS,AVENIDA DE CANILLEJAS A VICALVARO - CALLE DE A...,0,2018/9,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,TURISMO,CONDUCTOR,HOMBRE,IL,DE 25 A 29 AÑOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26458,2018-12-31,DE 21:00 A 21:59,LUNES,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA,0,2019/18,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,TURISMO,VIAJERO,HOMBRE,IL,DE 30 A 34 ANOS
26459,2018-12-31,DE 21:00 A 21:59,LUNES,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA,0,2019/18,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,AUTO-TAXI,CONDUCTOR,HOMBRE,IL,DE 40 A 44 AÑOS
26460,2018-12-31,DE 23:00 A 23:59,LUNES,HORTALEZA,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,0,2019/53,NO,NO,NO,...,NO,NO,SI,1,OTRAS CAUSAS,TURISMO,CONDUCTOR,MUJER,IL,DE 21 A 24 AÑOS
26461,2018-12-31,DE 23:00 A 23:59,LUNES,HORTALEZA,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,0,2019/53,NO,NO,NO,...,NO,NO,SI,1,OTRAS CAUSAS,TURISMO,CONDUCTOR,HOMBRE,HL,DE 50 A 54 AÑOS


In [18]:
total_accidents =pd.concat([clean_2016, clean_2017, clean_2018], axis =0)

In [19]:
total_accidents = total_accidents.reset_index(drop = True)

In [20]:
total_accidents

,fecha,rango_horario,dia_semana,distrito,lugar_accidente,nro,nro_parte,cpfa_granizo,cpfa_hielo,cpfa_lluvia,...,cpsv_grava_suelta,cpsv_hielo,cpsv_seca_y_limpia,nro_victimas,tipo_accidente,tipo_vehiculo,tipo_persona,sexo,lesividad,tramo_edad
0,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 6 A 9 AÑOS
1,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 40 A 44 AÑOS
2,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 35 A 39 AÑOS
3,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,NO ASIGNADO,PEATON,MUJER,HL,DE 35 A 39 AÑOS
4,2016-01-01,DE 00:00 A 00:59,VIERNES,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,NO,NO,NO,...,NO,NO,SI,4,ATROPELLO,TURISMO,CONDUCTOR,HOMBRE,IL,DE 30 A 34 ANOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78031,2018-12-31,DE 21:00 A 21:59,LUNES,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA,0,2019/18,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,TURISMO,VIAJERO,HOMBRE,IL,DE 30 A 34 ANOS
78032,2018-12-31,DE 21:00 A 21:59,LUNES,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA,0,2019/18,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,AUTO-TAXI,CONDUCTOR,HOMBRE,IL,DE 40 A 44 AÑOS
78033,2018-12-31,DE 23:00 A 23:59,LUNES,HORTALEZA,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,0,2019/53,NO,NO,NO,...,NO,NO,SI,1,OTRAS CAUSAS,TURISMO,CONDUCTOR,MUJER,IL,DE 21 A 24 AÑOS
78034,2018-12-31,DE 23:00 A 23:59,LUNES,HORTALEZA,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,0,2019/53,NO,NO,NO,...,NO,NO,SI,1,OTRAS CAUSAS,TURISMO,CONDUCTOR,HOMBRE,HL,DE 50 A 54 AÑOS


In [21]:
def translate(row):
    row = row.rename(columns={"fecha": "date", "rango_horario": "hour", "dia_semana": "week_day", "distrito": "district", "lugar_accidente": "street", "nro": "number", "nro_parte": "id_accident", "cpfa_granizo": "ac_hail", "cpfa_hielo": "ac_ice", "cpfa_lluvia": "ac_rain", "cpfa_niebla": "ac_fog", "cpfa_seco": "ac_dry", "cpfa_nieve": "ac_snow", "cpsv_mojada": "wet", "cpsv_aceite": "oil", "cpsv_barro": "mud", "cpsv_grava_suelta": "gravel", "cpsv_hielo": "ice", "cpsv_seca_y_limpia": "dry_and_clean", "nro_victimas": "number_of_victims", "tipo_accidente": "type_of_accident", "tipo_vehiculo": "vehicle", "tipo_persona": "type_of_person", "sexo": "gender", "lesividad": "gravity", "tramo_edad": "age"})
    pattern = 'DE '
    pattern1 = ' A '
    for i in range(len(row['hour'])):
        row['hour'][i] = re.sub(pattern, '',row['hour'][i])
        row['hour'][i] = re.sub(pattern1, '-',row['hour'][i])
    #row['hour'] = row['hour'].replace("DE","BETWEEN")
    #row['hour'] = row['hour'].replace("A","AND")
    row['week_day'] = row['week_day'].replace("LUNES","Monday")
    row['week_day'] = row['week_day'].replace("MARTES","Tuesday")
    row['week_day'] = row['week_day'].replace("MIERCOLES","Wednesday")
    row['week_day'] = row['week_day'].replace("JUEVES","Thursday")
    row['week_day'] = row['week_day'].replace("VIERNES","Friday")
    row['week_day'] = row['week_day'].replace("SABADO","Saturday")
    row['week_day'] = row['week_day'].replace("DOMINGO","Sunday")
    #I'm going to transform all this columns to numerical columns with a 0 if the answer is No and 1 if it is yes, and then i can sum the yes in tableau and obtain the different quantity of causes. 
    row['ac_hail'] = row['ac_hail'].replace("SI","1")
    row['ac_ice'] = row['ac_ice'].replace("SI","1")
    row['ac_rain'] = row['ac_rain'].replace("SI","1")
    row['ac_fog'] = row['ac_fog'].replace("SI","1")
    row['ac_dry'] = row['ac_dry'].replace("SI","1")
    row['ac_snow'] = row['ac_snow'].replace("SI","1")
    row['wet'] = row['wet'].replace("SI","1")
    row['oil'] = row['oil'].replace("SI","1")
    row['mud'] = row['mud'].replace("SI","1")
    row['gravel'] = row['gravel'].replace("SI","1")
    row['ice'] = row['ice'].replace("SI","1")
    row['dry_and_clean'] = row['dry_and_clean'].replace("SI","1")
    row['ac_hail'] = row['ac_hail'].replace("NO","0")
    row['ac_ice'] = row['ac_ice'].replace("NO","0")
    row['ac_rain'] = row['ac_rain'].replace("NO","0")
    row['ac_fog'] = row['ac_fog'].replace("NO","0")
    row['ac_dry'] = row['ac_dry'].replace("NO","0")
    row['ac_snow'] = row['ac_snow'].replace("NO","0")
    row['wet'] = row['wet'].replace("NO","0")
    row['oil'] = row['oil'].replace("NO","0")
    row['mud'] = row['mud'].replace("NO","0")
    row['gravel'] = row['gravel'].replace("NO","0")
    row['ice'] = row['ice'].replace("NO","0")
    row['dry_and_clean'] = row['dry_and_clean'].replace("NO","0")
    row['ac_hail'] = pd.to_numeric(row['ac_hail'])
    row['ac_ice'] = pd.to_numeric(row['ac_ice'])
    row['ac_rain'] = pd.to_numeric(row['ac_rain'])
    row['ac_fog'] = pd.to_numeric(row['ac_fog'])
    row['ac_dry'] = pd.to_numeric(row['ac_dry'])
    row['ac_snow'] = pd.to_numeric(row['ac_snow'])
    row['wet'] = pd.to_numeric(row['wet'])
    row['oil'] = pd.to_numeric(row['oil'])
    row['mud'] = pd.to_numeric(row['mud'])
    row['gravel'] = pd.to_numeric(row['gravel'])
    row['ice'] = pd.to_numeric(row['ice'])
    row['dry_and_clean'] = pd.to_numeric(row['dry_and_clean'])
    row['type_of_accident'] = row['type_of_accident'].replace("COLISIÓN DOBLE","multiple crash")
    row['type_of_accident'] = row['type_of_accident'].replace("ATROPELLO","run over")
    row['type_of_accident'] = row['type_of_accident'].replace("COLISIÓN MÚLTIPLE","multiple crash")
    row['type_of_accident'] = row['type_of_accident'].replace("CHOQUE CON OBJETO FIJO","crash with a fixed object")
    row['type_of_accident'] = row['type_of_accident'].replace("CAÍDA MOTOCICLETA","motorcycle crash")
    row['type_of_accident'] = row['type_of_accident'].replace("CAÍDA VIAJERO BUS","other")
    row['type_of_accident'] = row['type_of_accident'].replace("CAÍDA BICICLETA","other")
    row['type_of_accident'] = row['type_of_accident'].replace("CAÍDA CICLOMOTOR","other")
    row['type_of_accident'] = row['type_of_accident'].replace("OTRAS CAUSAS","other")
    row['type_of_accident'] = row['type_of_accident'].replace("VUELCO","other")
    row['type_of_accident'] = row['type_of_accident'].replace("CAÍDA VEHÍCULO 3 RUEDAS","other")
    row['vehicle'] = row['vehicle'].replace("TURISMO","car")
    row['vehicle'] = row['vehicle'].replace("MOTOCICLETA","motorcycle")
    row['vehicle'] = row['vehicle'].replace("NO ASIGNADO","unknown")
    row['vehicle'] = row['vehicle'].replace("FURGONETA","van")
    row['vehicle'] = row['vehicle'].replace("AUTOBUS-AUTOCAR","bus")
    row['vehicle'] = row['vehicle'].replace("AUTO-TAXI","car")
    row['vehicle'] = row['vehicle'].replace("BICICLETA","bycicle")
    row['vehicle'] = row['vehicle'].replace("CICLOMOTOR","other")
    row['vehicle'] = row['vehicle'].replace("CAMION","other")
    row['vehicle'] = row['vehicle'].replace("VARIOS","other")
    row['vehicle'] = row['vehicle'].replace("AMBULANCIA","other")
    row['vehicle'] = row['vehicle'].replace("VEH.3 RUEDAS","other")
    row['type_of_person'] = row['type_of_person'].replace("PEATON","pedestrian")
    row['type_of_person'] = row['type_of_person'].replace("CONDUCTOR","driver")
    row['type_of_person'] = row['type_of_person'].replace("VIAJERO","traveler")
    row['type_of_person'] = row['type_of_person'].replace("PEATON","pedestrian")
    row['gender'] = row['gender'].replace("HOMBRE","male")
    row['gender'] = row['gender'].replace("MUJER","female")
    row['gender'] = row['gender'].replace("NO ASIGNADO","unknown")
    row['gravity'] = row['gravity'].replace("HL","mild injury")
    row['gravity'] = row['gravity'].replace("IL","uninjured")
    row['gravity'] = row['gravity'].replace("NO ASIGNADA","unknown")
    row['gravity'] = row['gravity'].replace("HG","serious injury")
    row['gravity'] = row['gravity'].replace("MT","death")
    row['age'] = row['age'].replace("DESCONOCIDA","unknown")
    pattern2 = 'MAS '
    pattern3 = 'A[NÑ]OS'
    for i in range(len(row['hour'])):
        if row['age'][i] == 'unknown':
            row['age'][i] = row['age'][i]
        else:
            row['age'][i] = re.sub(pattern2, '+',row['age'][i])
            row['age'][i] = re.sub(pattern, '',row['age'][i])
            row['age'][i] = re.sub(pattern1, '-',row['age'][i])
            row['age'][i] = re.sub(pattern3, 'years old',row['age'][i])
    #row['age'] = row['age'].replace("DE","BETWEEN")
    #row['age'] = row['age'].replace("A","AND")
    #row['age'] = row['age'].replace("AÑOS","years old")
    return row

In [22]:
clean_accidents = translate(total_accidents)
clean_accidents

<ipython-input-21-a3ecc0faabb8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['hour'][i] = re.sub(pattern, '',row['hour'][i])
<ipython-input-21-a3ecc0faabb8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['hour'][i] = re.sub(pattern1, '-',row['hour'][i])
<ipython-input-21-a3ecc0faabb8>:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['age'][i] = re.sub(pattern2, '+',row['age'][i])
<ipython-input-21-a3ecc0faabb8>:97: S

,date,hour,week_day,district,street,number,id_accident,ac_hail,ac_ice,ac_rain,...,gravel,ice,dry_and_clean,number_of_victims,type_of_accident,vehicle,type_of_person,gender,gravity,age
0,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,0,0,0,...,0,0,1,4,run over,unknown,pedestrian,male,mild injury,6-9 years old
1,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,0,0,0,...,0,0,1,4,run over,unknown,pedestrian,male,mild injury,40-44 years old
2,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,0,0,0,...,0,0,1,4,run over,unknown,pedestrian,male,mild injury,35-39 years old
3,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,0,0,0,...,0,0,1,4,run over,unknown,pedestrian,female,mild injury,35-39 years old
4,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,0,0,0,...,0,0,1,4,run over,car,driver,male,uninjured,30-34 years old
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78031,2018-12-31,21:00-21:59,Monday,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA,0,2019/18,0,0,0,...,0,0,1,1,multiple crash,car,traveler,male,uninjured,30-34 years old
78032,2018-12-31,21:00-21:59,Monday,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA,0,2019/18,0,0,0,...,0,0,1,1,multiple crash,car,driver,male,uninjured,40-44 years old
78033,2018-12-31,23:00-23:59,Monday,HORTALEZA,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,0,2019/53,0,0,0,...,0,0,1,1,other,car,driver,female,uninjured,21-24 years old
78034,2018-12-31,23:00-23:59,Monday,HORTALEZA,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,0,2019/53,0,0,0,...,0,0,1,1,other,car,driver,male,mild injury,50-54 years old


In [23]:
for col in clean_accidents.columns:
    print(clean_accidents[col].unique())

['2016-01-01T00:00:00.000000000' '2016-01-02T00:00:00.000000000'
 '2016-01-03T00:00:00.000000000' ... '2018-12-29T00:00:00.000000000'
 '2018-12-30T00:00:00.000000000' '2018-12-31T00:00:00.000000000']
['00:00-00:59' '1:00-1:59' '2:00-2:59' '3:00-3:59' '5:00-5:59' '6:00-6:59'
 '9:00-9:59' '11:00-11:59' '12:00-12:59' '14:00-14:59' '16:00-16:59'
 '18:00-18:59' '19:00-19:59' '20:00-20:59' '21:00-21:59' '22:00-22:59'
 '23:00-23:59' '15:00-15:59' '17:00-17:59' '10:00-10:59' '13:00-13:59'
 '7:00-7:59' '8:00-8:59' '4:00-4:59']
['Friday' 'Saturday' 'Sunday' 'Monday' 'Tuesday' 'Wednesday' 'Thursday']
['PUENTE DE VALLECAS' 'CIUDAD LINEAL' 'CHAMARTIN' 'USERA' 'LATINA'
 'RETIRO' 'VILLAVERDE' 'SALAMANCA' 'CHAMBERI' 'ARGANZUELA' 'TETUAN'
 'FUENCARRAL-EL PARDO' 'HORTALEZA' 'CENTRO' 'BARAJAS' 'CARABANCHEL'
 'VILLA DE VALLECAS' 'SAN BLAS' 'MONCLOA-ARAVACA' 'VICALVARO' 'MORATALAZ']
['AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE DARIEN'
 'AVENIDA DEL MARQUES DE CORBERA NUM' 'AUTOVIA  M-30 CALZADA 1 KM.' ..

In [24]:
clean_accidents.isna().sum()

date                 0
hour                 0
week_day             0
district             0
street               0
number               0
id_accident          0
ac_hail              0
ac_ice               0
ac_rain              0
ac_fog               0
ac_dry               0
ac_snow              0
wet                  0
oil                  0
mud                  0
gravel               0
ice                  0
dry_and_clean        0
number_of_victims    0
type_of_accident     0
vehicle              0
type_of_person       0
gender               0
gravity              0
age                  0
dtype: int64

In [25]:
# I'm going to check if all unknown vehicles are because type of people are pedestrians

In [26]:
clean_accidents['vehicle'].value_counts()

car           47977
motorcycle    12865
unknown        5208
other          3674
van            3476
bus            2624
bycicle        2212
Name: vehicle, dtype: int64

In [27]:
clean_accidents['type_of_person'].value_counts()

driver        57013
traveler      15858
pedestrian     5165
Name: type_of_person, dtype: int64

In [28]:
# Not all, but probably most of them

In [29]:
for i in range(len(clean_accidents['vehicle'])):
    if (clean_accidents['vehicle'][i] == 'unknown') & (clean_accidents['type_of_person'][i] == 'pedestrian'):
        clean_accidents['vehicle'][i] = 'none'

<ipython-input-29-6e9585f11168>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_accidents['vehicle'][i] = 'none'


In [30]:
clean_accidents['vehicle'].value_counts()

car           47977
motorcycle    12865
none           5159
other          3674
van            3476
bus            2624
bycicle        2212
unknown          49
Name: vehicle, dtype: int64

In [31]:
pedestrian = clean_accidents[clean_accidents['type_of_person'] =='pedestrian']
pedestrian['vehicle'].value_counts()

none     5159
other       3
car         2
van         1
Name: vehicle, dtype: int64

In [32]:
# If they are pedestrians, it is impossible they were on a car or a van, so i'm going to change it. Other it's possible because they can have an skate or something like that.

In [33]:
for i in range(len(clean_accidents['vehicle'])):
    if ((clean_accidents['vehicle'][i] == 'car') or (clean_accidents['vehicle'][i] == 'van')) & (clean_accidents['type_of_person'][i] == 'pedestrian'):
        clean_accidents['vehicle'][i] = 'none'

<ipython-input-33-5d4695f3e438>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_accidents['vehicle'][i] = 'none'


In [34]:
pedestrian = clean_accidents[clean_accidents['type_of_person'] =='pedestrian']
pedestrian['vehicle'].value_counts()

none     5162
other       3
Name: vehicle, dtype: int64

In [35]:
codes = pd.read_excel('./files_for_lab/codes.xlsx')
codes

,DISTRICT,CODE
0,USERA,12
1,SAN BLAS,20
2,FUENCARRAL-EL PARDO,8
3,ARGANZUELA,2
4,SALAMANCA,4
5,CARABANCHEL,11
6,TETUAN,6
7,CHAMBERI,7
8,VILLAVERDE,17
9,PUENTE DE VALLECAS,13


In [36]:
#I'm going to add a new column with the district code

In [37]:
clean_accidents['district_code'] = 0
for r in range(len(codes['DISTRICT'])):
    for i in range(len(clean_accidents['district'])):
        if clean_accidents['district'][i] == codes['DISTRICT'][r]:
            clean_accidents['district_code'][i] += codes['CODE'][r]

<ipython-input-37-c437499abf76>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_accidents['district_code'][i] += codes['CODE'][r]


In [38]:
clean_accidents

,date,hour,week_day,district,street,number,id_accident,ac_hail,ac_ice,ac_rain,...,ice,dry_and_clean,number_of_victims,type_of_accident,vehicle,type_of_person,gender,gravity,age,district_code
0,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,0,0,0,...,0,1,4,run over,none,pedestrian,male,mild injury,6-9 years old,13
1,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,0,0,0,...,0,1,4,run over,none,pedestrian,male,mild injury,40-44 years old,13
2,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,0,0,0,...,0,1,4,run over,none,pedestrian,male,mild injury,35-39 years old,13
3,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,0,0,0,...,0,1,4,run over,none,pedestrian,female,mild injury,35-39 years old,13
4,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,0,0,0,...,0,1,4,run over,car,driver,male,uninjured,30-34 years old,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78031,2018-12-31,21:00-21:59,Monday,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA,0,2019/18,0,0,0,...,0,1,1,multiple crash,car,traveler,male,uninjured,30-34 years old,4
78032,2018-12-31,21:00-21:59,Monday,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA,0,2019/18,0,0,0,...,0,1,1,multiple crash,car,driver,male,uninjured,40-44 years old,4
78033,2018-12-31,23:00-23:59,Monday,HORTALEZA,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,0,2019/53,0,0,0,...,0,1,1,other,car,driver,female,uninjured,21-24 years old,16
78034,2018-12-31,23:00-23:59,Monday,HORTALEZA,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,0,2019/53,0,0,0,...,0,1,1,other,car,driver,male,mild injury,50-54 years old,16


In [39]:
max(clean_accidents['district_code'])

21

In [40]:
clean_accidents.dtypes

date                 datetime64[ns]
hour                         object
week_day                     object
district                     object
street                       object
number                       object
id_accident                  object
ac_hail                       int64
ac_ice                        int64
ac_rain                       int64
ac_fog                        int64
ac_dry                        int64
ac_snow                       int64
wet                           int64
oil                           int64
mud                           int64
gravel                        int64
ice                           int64
dry_and_clean                 int64
number_of_victims             int64
type_of_accident             object
vehicle                      object
type_of_person               object
gender                       object
gravity                      object
age                          object
district_code                 int64
dtype: object

In [41]:
clean_accidents['number']

0        0
1        0
2        0
3        0
4        0
        ..
78031    0
78032    0
78033    0
78034    0
78035    0
Name: number, Length: 78036, dtype: object

In [42]:
# Transforming the number column to a int.
for i in range(len(clean_accidents['number'])):
    clean_accidents['number'][i] = str(clean_accidents['number'][i])
    clean_accidents['number'][i] = clean_accidents['number'][i].strip()
clean_accidents['number'] = pd.to_numeric(clean_accidents['number'])

<ipython-input-42-dc1280025096>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_accidents['number'][i] = str(clean_accidents['number'][i])
<ipython-input-42-dc1280025096>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_accidents['number'][i] = clean_accidents['number'][i].strip()


In [43]:
clean_accidents['number'].value_counts(dropna=False)

0.0        36276
1.0         2189
2.0         1603
NaN         1168
3.0         1048
           ...  
15300.0        1
25600.0        1
22500.0        1
386.0          1
670.0          1
Name: number, Length: 755, dtype: int64

In [44]:
clean_accidents['number'] = clean_accidents['number'].fillna(0)

In [45]:
clean_accidents['number'].value_counts(dropna=False)

0.0        37444
1.0         2189
2.0         1603
3.0         1048
4.0          988
           ...  
3100.0         1
23100.0        1
625.0          1
612.0          1
22500.0        1
Name: number, Length: 754, dtype: int64

In [46]:
# This is necessary because I need %street% works in MYSQL
for i in range(len(clean_accidents['street'])):
    clean_accidents['street'][i] = clean_accidents['street'][i] + '-'

<ipython-input-46-da19bcaacab8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_accidents['street'][i] = clean_accidents['street'][i] + '-'


In [47]:
clean_accidents['street']

0        AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...
1        AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...
2        AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...
3        AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...
4        AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...
                               ...                        
78031            CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA-
78032            CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA-
78033    CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...
78034    CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...
78035    CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...
Name: street, Length: 78036, dtype: object

In [48]:
accidents = clean_accidents[['date', 'hour' , 'week_day', 'district', 'street','number', 'id_accident', 'number_of_victims', 'type_of_accident', 'vehicle', 'type_of_person',
       'gender', 'gravity', 'age', 'district_code']]

In [49]:
#accidents.to_excel('./files_for_lab/accidents.xlsx')
#accidents.to_csv('./files_for_lab/accidents.csv')

In [50]:
# I'm going to use this file to obtain postal codes in MYSQL using the postal_codes file.

In [51]:
postal_codes = pd.read_csv('./files_for_lab/postal_codes.csv', sep=';', engine='python')
postal_codes

,Codigo de via,Clase de la via,Particula de la via,Nombre de la via,Tipo de tramo,Nombre de la aproximacion,Numero inicial del tramo,Calificador del numero inicial del tramo,Numero final del tramo,Calificador del numero final del tramo,Codigo postal,Seccion de carteria
0,127,CALLE,DE,ISABEL COLBRAND,par,NUM,2,,12,,28050,N/D
1,127,CALLE,DE,ISABEL COLBRAND,par,NUM,14,,22,,28050,N/D
2,127,CALLE,DE,ISABEL COLBRAND,par,NUM,24,,9998,,28050,N/D
3,150,CALLE,DE,GONZALEZ DAVILA,impar,NUM,1,,9999,,28031,N/D
4,150,CALLE,DE,GONZALEZ DAVILA,par,NUM,2,,9998,,28031,N/D
...,...,...,...,...,...,...,...,...,...,...,...,...
22916,31004755,TRAVESIA,DE,CAPRI,par,NUM,2,,9998,,28032,N/D
22917,31004759,CALLE,DE,ANGELES GARCIA MADRID,impar,NUM,1,,9999,,28007,N/D
22918,31004759,CALLE,DE,ANGELES GARCIA MADRID,par,NUM,2,,9998,,28007,N/D
22919,31004764,PLAZA,DE LOS,CHISPEROS DE CHAMBERI,impar,NUM,1,,9999,,28010,N/D


In [52]:
# I'm not going to translate this table because it's not important. I'm going to use this table and street_codes to obtain the postal code.

In [53]:
postal_codes['Codigo postal'].nunique()

56

In [54]:
postal_code_finished = postal_codes[['Codigo de via', 'Clase de la via','Particula de la via', 'Nombre de la via', 'Numero inicial del tramo', 'Numero final del tramo', 'Codigo postal']]
postal_code_finished

,Codigo de via,Clase de la via,Particula de la via,Nombre de la via,Numero inicial del tramo,Numero final del tramo,Codigo postal
0,127,CALLE,DE,ISABEL COLBRAND,2,12,28050
1,127,CALLE,DE,ISABEL COLBRAND,14,22,28050
2,127,CALLE,DE,ISABEL COLBRAND,24,9998,28050
3,150,CALLE,DE,GONZALEZ DAVILA,1,9999,28031
4,150,CALLE,DE,GONZALEZ DAVILA,2,9998,28031
...,...,...,...,...,...,...,...
22916,31004755,TRAVESIA,DE,CAPRI,2,9998,28032
22917,31004759,CALLE,DE,ANGELES GARCIA MADRID,1,9999,28007
22918,31004759,CALLE,DE,ANGELES GARCIA MADRID,2,9998,28007
22919,31004764,PLAZA,DE LOS,CHISPEROS DE CHAMBERI,1,9999,28010


In [55]:
code_postal_code = pd.read_excel('./files_for_lab/code_postal_code.xlsx')
code_postal_code

,Clase de la via,Particula de la via,Nombre de la via,Codigo de vía,Cantidad de numeraciones en la via,Distritos atravesados,Codigos postales,Campo que indica si tiene zonas en S E R,Tipo de la via,Situacion de la via respecto al terreno,Tipo de denominación de la vía,Superficie aproximada m2 (aceras-isletas),Superficie aproximada m2 (calzada),Literal completo del vial
0,CALLE,DE,ISABEL COLBRAND,127,52,8,28050,NO,Vía,Nivel,Pleno,11858,16916,CALLE DE ISABEL COLBRAND ...
1,CALLE,DE,GONZALEZ DAVILA,150,18,18,varios,NO,Vía,Nivel,Pleno,6655,13875,CALLE DE GONZÁLEZ DÁVILA ...
2,CALLE,DE LA,ABADA,200,12,2021-01-01 00:00:00,varios,SI,Vía,Nivel,Pleno,2145,0,CALLE DE LA ABADA ...
3,CALLE,DE LOS,ABADES,300,19,1,varios,SI,Vía,Nivel,Pleno,897,0,CALLE DE LOS ABADES ...
4,CALLE,DE LA,ABADESA,400,7,6,varios,SI,Vía,Nivel,Pleno,131,314,CALLE DE LA ABADESA ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9846,CAÑADA,,REAL SECTOR 5,31005678,140,19,,NO,Topónimo,Nivel,Admon,0,0,CAÑADA REAL SECTOR 5 ...
9847,CAÑADA,,REAL SECTOR 3,31005679,48,19,,NO,Topónimo,Nivel,Admon,0,0,CAÑADA REAL SECTOR 3 ...
9848,CAÑADA,,REAL SECTOR 4,31005680,89,19,,NO,Topónimo,Nivel,Pleno,0,0,CAÑADA REAL SECTOR 4 ...
9849,CALLE,DE,ISIDORO ALVAREZ ALVAREZ,31005685,10,2,,SI,Vía,Nivel,Pleno,0,0,CALLE DE ISIDORO ÁLVAREZ ÁLVAREZ ...


In [56]:
code_postal_code.columns

Index(['Clase de la via', 'Particula de la via', 'Nombre de la via',
       'Codigo de vía', 'Cantidad de numeraciones en la via',
       'Distritos atravesados', 'Codigos postales',
       'Campo que indica si tiene zonas en S E R ', 'Tipo de la via',
       'Situacion de la via respecto al terreno',
       'Tipo de denominación de la vía ',
       'Superficie aproximada m2 (aceras-isletas)',
       'Superficie aproximada m2 (calzada)', 'Literal completo del vial '],
      dtype='object')

In [57]:
code_postal_code_finished = code_postal_code[['Clase de la via', 'Particula de la via', 'Nombre de la via', 'Codigo de vía', 'Distritos atravesados', 'Literal completo del vial ']]
code_postal_code_finished

,Clase de la via,Particula de la via,Nombre de la via,Codigo de vía,Distritos atravesados,Literal completo del vial
0,CALLE,DE,ISABEL COLBRAND,127,8,CALLE DE ISABEL COLBRAND ...
1,CALLE,DE,GONZALEZ DAVILA,150,18,CALLE DE GONZÁLEZ DÁVILA ...
2,CALLE,DE LA,ABADA,200,2021-01-01 00:00:00,CALLE DE LA ABADA ...
3,CALLE,DE LOS,ABADES,300,1,CALLE DE LOS ABADES ...
4,CALLE,DE LA,ABADESA,400,6,CALLE DE LA ABADESA ...
...,...,...,...,...,...,...
9846,CAÑADA,,REAL SECTOR 5,31005678,19,CAÑADA REAL SECTOR 5 ...
9847,CAÑADA,,REAL SECTOR 3,31005679,19,CAÑADA REAL SECTOR 3 ...
9848,CAÑADA,,REAL SECTOR 4,31005680,19,CAÑADA REAL SECTOR 4 ...
9849,CALLE,DE,ISIDORO ALVAREZ ALVAREZ,31005685,2,CALLE DE ISIDORO ÁLVAREZ ÁLVAREZ ...


In [58]:
code_postal_code_finished.dtypes

Clase de la via               object
Particula de la via           object
Nombre de la via              object
Codigo de vía                  int64
Distritos atravesados         object
Literal completo del vial     object
dtype: object

In [59]:
code_postal_code_finished['Distritos atravesados'].unique()

array([8, 18, datetime.datetime(2021, 1, 1, 0, 0), 1, 6, 15, 20,
       datetime.datetime(2021, 7, 4, 0, 0), 16, 3, 5, 2, 9, 14, 21,
       datetime.datetime(2021, 9, 8, 0, 0), 13,
       '16-20                                                       ',
       '08-16-21                                                    ', 17,
       '16-21                                                       ', 12,
       7, 10, datetime.datetime(2021, 8, 4, 0, 0),
       '14-15                                                       ',
       '08-15-16-20-21                                              ',
       datetime.datetime(2017, 12, 1, 0, 0), 11,
       datetime.datetime(2018, 8, 1, 0, 0),
       datetime.datetime(2018, 9, 1, 0, 0),
       datetime.datetime(2016, 8, 1, 0, 0),
       datetime.datetime(2021, 12, 11, 0, 0),
       '                                                            ',
       datetime.datetime(2021, 8, 1, 0, 0),
       datetime.datetime(2021, 8, 7, 0, 0),
       datetime.dat

In [60]:
# This column is one of the most important because it's a link between the two tables i need in mysql to obtain the postal code. I need to clean it.
# Datetime is an error, the day or the month are the different districts. Day is the main district and month a district where the street pass but not a lot. In day it's the most part of the street.
# And there are a lot of streets with many code districts, so i'm going to use random to pick one because i need unique values.

In [61]:
import random
import time
import datetime
from datetime import date

In [62]:
for i in range(len(code_postal_code_finished['Distritos atravesados'])):
    if isinstance( code_postal_code_finished['Distritos atravesados'][i], str ) == True:
        code_postal_code_finished['Distritos atravesados'][i] = code_postal_code_finished['Distritos atravesados'][i].strip()

<ipython-input-62-10452abc5287>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  code_postal_code_finished['Distritos atravesados'][i] = code_postal_code_finished['Distritos atravesados'][i].strip()
C:\Users\Propietario\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [63]:
for i in range(len(code_postal_code_finished['Distritos atravesados'])):
    if code_postal_code_finished['Distritos atravesados'][i] == '':
        code_postal_code_finished['Distritos atravesados'] = code_postal_code_finished['Distritos atravesados']
    else:
        if (isinstance( code_postal_code_finished['Distritos atravesados'][i], str ) == False) & (isinstance( code_postal_code_finished['Distritos atravesados'][i], int ) == False):
            day = code_postal_code_finished['Distritos atravesados'][i].day
            code_postal_code_finished['Distritos atravesados'][i] = day
        else:
            if isinstance( code_postal_code_finished['Distritos atravesados'][i], int ) == True:
                code_postal_code_finished['Distritos atravesados'][i] = code_postal_code_finished['Distritos atravesados'][i]
            else:
                pattern = '\d+'
                list_ = re.findall(pattern, code_postal_code_finished['Distritos atravesados'][i])
                winner = random.choice(list_)
                code_postal_code_finished['Distritos atravesados'][i] = winner
code_postal_code_finished['Distritos atravesados'] = pd.to_numeric(code_postal_code_finished['Distritos atravesados'])

<ipython-input-63-77a26863843f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  code_postal_code_finished['Distritos atravesados'][i] = code_postal_code_finished['Distritos atravesados'][i]
<ipython-input-63-77a26863843f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  code_postal_code_finished['Distritos atravesados'][i] = day
<ipython-input-63-77a26863843f>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  code_postal_code_finishe

In [64]:
code_postal_code_finished

,Clase de la via,Particula de la via,Nombre de la via,Codigo de vía,Distritos atravesados,Literal completo del vial
0,CALLE,DE,ISABEL COLBRAND,127,8.0,CALLE DE ISABEL COLBRAND ...
1,CALLE,DE,GONZALEZ DAVILA,150,18.0,CALLE DE GONZÁLEZ DÁVILA ...
2,CALLE,DE LA,ABADA,200,1.0,CALLE DE LA ABADA ...
3,CALLE,DE LOS,ABADES,300,1.0,CALLE DE LOS ABADES ...
4,CALLE,DE LA,ABADESA,400,6.0,CALLE DE LA ABADESA ...
...,...,...,...,...,...,...
9846,CAÑADA,,REAL SECTOR 5,31005678,19.0,CAÑADA REAL SECTOR 5 ...
9847,CAÑADA,,REAL SECTOR 3,31005679,19.0,CAÑADA REAL SECTOR 3 ...
9848,CAÑADA,,REAL SECTOR 4,31005680,19.0,CAÑADA REAL SECTOR 4 ...
9849,CALLE,DE,ISIDORO ALVAREZ ALVAREZ,31005685,2.0,CALLE DE ISIDORO ÁLVAREZ ÁLVAREZ ...


In [65]:
code_postal_code_finished['Distritos atravesados'].unique()

array([ 8., 18.,  1.,  6., 15., 20.,  4., 16.,  3.,  5.,  2.,  9., 14.,
       21., 13., 17., 12.,  7., 10., 11., nan, 19.])

In [66]:
# The NAN aare the empty spaces, so i can drop all these rows

In [67]:
code_postal_code_finished['Distritos atravesados'].value_counts(dropna = False)

8.0     850
9.0     825
11.0    748
1.0     730
16.0    660
13.0    640
20.0    567
10.0    510
5.0     497
15.0    492
6.0     429
17.0    421
18.0    394
12.0    373
21.0    311
19.0    285
2.0     270
4.0     268
3.0     261
7.0     204
14.0    114
NaN       2
Name: Distritos atravesados, dtype: int64

In [68]:
code_postal_code_finished = code_postal_code_finished.dropna()
code_postal_code_finished = code_postal_code_finished.reset_index(drop=True)
code_postal_code_finished

,Clase de la via,Particula de la via,Nombre de la via,Codigo de vía,Distritos atravesados,Literal completo del vial
0,CALLE,DE,ISABEL COLBRAND,127,8.0,CALLE DE ISABEL COLBRAND ...
1,CALLE,DE,GONZALEZ DAVILA,150,18.0,CALLE DE GONZÁLEZ DÁVILA ...
2,CALLE,DE LA,ABADA,200,1.0,CALLE DE LA ABADA ...
3,CALLE,DE LOS,ABADES,300,1.0,CALLE DE LOS ABADES ...
4,CALLE,DE LA,ABADESA,400,6.0,CALLE DE LA ABADESA ...
...,...,...,...,...,...,...
9844,CAÑADA,,REAL SECTOR 5,31005678,19.0,CAÑADA REAL SECTOR 5 ...
9845,CAÑADA,,REAL SECTOR 3,31005679,19.0,CAÑADA REAL SECTOR 3 ...
9846,CAÑADA,,REAL SECTOR 4,31005680,19.0,CAÑADA REAL SECTOR 4 ...
9847,CALLE,DE,ISIDORO ALVAREZ ALVAREZ,31005685,2.0,CALLE DE ISIDORO ÁLVAREZ ÁLVAREZ ...


In [69]:
postal_code_finished['Codigo postal'].value_counts(dropna=False)

28023    811
28011    801
28021    793
28042    722
28043    677
28019    675
28053    655
28039    628
28033    625
28025    624
28022    624
28035    613
28002    604
28041    574
28026    560
28037    560
28027    550
28031    547
28017    526
28034    517
28018    499
28005    498
28038    474
28029    451
28032    433
28044    426
28028    424
28045    417
28050    395
28007    335
28012    321
28055    321
28004    318
28016    308
28051    293
28054    292
28030    288
28049    287
28047    278
28013    267
28020    266
28024    243
28052    235
28015    219
28010    218
28036    211
28003    207
28008    207
28014    199
28009    191
28006    177
28040    176
28048    151
28001    134
28046     72
28760      4
Name: Codigo postal, dtype: int64

In [70]:
def deleting_commas(row):
    for i in range(len(row['Nombre de la via'])):
        row['Nombre de la via'][i] = row['Nombre de la via'][i].replace(",","-")
    return row

In [71]:
postal_code_finished = deleting_commas(postal_code_finished)

<ipython-input-70-cc3ee0bed7a9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['Nombre de la via'][i] = row['Nombre de la via'][i].replace(",","-")
<ipython-input-71-7aa09d19ab1c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  postal_code_finished = deleting_commas(postal_code_finished)


In [72]:
code_postal_code_finished = deleting_commas(code_postal_code_finished)

<ipython-input-70-cc3ee0bed7a9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['Nombre de la via'][i] = row['Nombre de la via'][i].replace(",","-")


In [73]:
for i in range(len(code_postal_code_finished['Literal completo del vial '])):
    code_postal_code_finished['Literal completo del vial '][i] = code_postal_code_finished['Literal completo del vial '][i].replace(",","-")

<ipython-input-73-df1b564ee1d7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  code_postal_code_finished['Literal completo del vial '][i] = code_postal_code_finished['Literal completo del vial '][i].replace(",","-")


In [74]:
postal_code_finished = postal_code_finished.drop_duplicates()

In [75]:
postal_code_finished = postal_code_finished.reset_index(drop=True)

In [76]:
postal_code_finished

,Codigo de via,Clase de la via,Particula de la via,Nombre de la via,Numero inicial del tramo,Numero final del tramo,Codigo postal
0,127,CALLE,DE,ISABEL COLBRAND,2,12,28050
1,127,CALLE,DE,ISABEL COLBRAND,14,22,28050
2,127,CALLE,DE,ISABEL COLBRAND,24,9998,28050
3,150,CALLE,DE,GONZALEZ DAVILA,1,9999,28031
4,150,CALLE,DE,GONZALEZ DAVILA,2,9998,28031
...,...,...,...,...,...,...,...
22916,31004755,TRAVESIA,DE,CAPRI,2,9998,28032
22917,31004759,CALLE,DE,ANGELES GARCIA MADRID,1,9999,28007
22918,31004759,CALLE,DE,ANGELES GARCIA MADRID,2,9998,28007
22919,31004764,PLAZA,DE LOS,CHISPEROS DE CHAMBERI,1,9999,28010


In [77]:
# This part is to can compare different tables using LIKE in MYSQL
def add_percentage(row):
    for i in range(len(row['Nombre de la via'])):
        row['Clase de la via'][i] = row['Clase de la via'][i].strip()
        row['Clase de la via'][i] = '%' + row['Clase de la via'][i] + '%'
        row['Nombre de la via'][i] = row['Nombre de la via'][i].strip()
        row['Nombre de la via'][i] = '%' + row['Nombre de la via'][i] + '%'
    return row

In [78]:
postal_code_finished = add_percentage(postal_code_finished)

<ipython-input-77-1efaa02b6791>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['Clase de la via'][i] = row['Clase de la via'][i].strip()
<ipython-input-77-1efaa02b6791>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['Clase de la via'][i] = '%' + row['Clase de la via'][i] + '%'
<ipython-input-77-1efaa02b6791>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['Nombre de la via'][i] = row['Nombre de la via'][i].strip()
<ipy

In [79]:
postal_code_finished

,Codigo de via,Clase de la via,Particula de la via,Nombre de la via,Numero inicial del tramo,Numero final del tramo,Codigo postal
0,127,%CALLE%,DE,%ISABEL COLBRAND%,2,12,28050
1,127,%CALLE%,DE,%ISABEL COLBRAND%,14,22,28050
2,127,%CALLE%,DE,%ISABEL COLBRAND%,24,9998,28050
3,150,%CALLE%,DE,%GONZALEZ DAVILA%,1,9999,28031
4,150,%CALLE%,DE,%GONZALEZ DAVILA%,2,9998,28031
...,...,...,...,...,...,...,...
22916,31004755,%TRAVESIA%,DE,%CAPRI%,2,9998,28032
22917,31004759,%CALLE%,DE,%ANGELES GARCIA MADRID%,1,9999,28007
22918,31004759,%CALLE%,DE,%ANGELES GARCIA MADRID%,2,9998,28007
22919,31004764,%PLAZA%,DE LOS,%CHISPEROS DE CHAMBERI%,1,9999,28010


In [80]:
#postal_code_finished.to_csv('./files_for_lab/postal_code_finished.csv')
#code_postal_code_finished.to_csv('./files_for_lab/code_postal_code_finished.csv')

In [81]:
password = getpass.getpass()

 ········


In [82]:
# READING THE DATA
connection_string = 'mysql+pymysql://root:' + password + '@localhost/bank'
engine = create_engine(connection_string)
data = pd.read_sql('''SELECT a.*, 
CASE
WHEN a.week_day IN ('Monday','Tuesday','Wednesday','Thursday') THEN 'weekday'
ELSE 'weekend'
END AS 'weekday_or_weekend', pc.postal_code AS postal_code
FROM mid_camp_project.accidents a
INNER JOIN mid_camp_project.code_postal_code cpc USING(id_district)
INNER JOIN mid_camp_project.postal_code pc USING(id_street)
WHERE a.street LIKE pc.street_name
GROUP BY a.id;
''', engine)

In [83]:
data.head(10)

,id,date,hour,week_day,district,street,number,id_accident,number_of_victims,type_of_accident,vehicle,type_of_person,gender,gravity,age,id_district,weekday_or_weekend,postal_code
0,0,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,4,run over,none,pedestrian,male,mild injury,6-9 years old,13,weekend,28018
1,1,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,4,run over,none,pedestrian,male,mild injury,40-44 years old,13,weekend,28018
2,2,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,4,run over,none,pedestrian,male,mild injury,35-39 years old,13,weekend,28018
3,3,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,4,run over,none,pedestrian,female,mild injury,35-39 years old,13,weekend,28018
4,4,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,4,run over,car,driver,male,uninjured,30-34 years old,13,weekend,28018
5,26,2016-01-01,6:00-6:59,Friday,PUENTE DE VALLECAS,AVENIDA DE LA ALBUFERA - CALLE DE SIERRA CARBO...,0,2016/9,1,run over,none,pedestrian,female,serious injury,18-20 years old,13,weekend,28031
6,27,2016-01-01,6:00-6:59,Friday,PUENTE DE VALLECAS,AVENIDA DE LA ALBUFERA - CALLE DE SIERRA CARBO...,0,2016/9,1,run over,car,driver,male,uninjured,35-39 years old,13,weekend,28031
7,29,2016-01-01,11:00-11:59,Friday,PUENTE DE VALLECAS,AVENIDA DE BUENOS AIRES - AVENIDA DE ENTREVIAS -,0,2016/384,1,crash with a fixed object,car,driver,male,mild injury,25-29 years old,13,weekend,28053
8,38,2016-01-01,19:00-19:59,Friday,ARGANZUELA,CALLE DE LA BATALLA DEL SALADO NUM -,3,2016/36,2,multiple crash,car,driver,female,uninjured,45-49 years old,2,weekend,28045
9,39,2016-01-01,19:00-19:59,Friday,ARGANZUELA,CALLE DE LA BATALLA DEL SALADO NUM -,3,2016/36,2,multiple crash,car,driver,male,mild injury,25-29 years old,2,weekend,28045


In [84]:
data.shape

(59916, 18)

In [85]:
#data.to_excel('./files_for_lab/data.xlsx')

In [86]:
clean_accidents

,date,hour,week_day,district,street,number,id_accident,ac_hail,ac_ice,ac_rain,...,ice,dry_and_clean,number_of_victims,type_of_accident,vehicle,type_of_person,gender,gravity,age,district_code
0,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0.0,2016/4,0,0,0,...,0,1,4,run over,none,pedestrian,male,mild injury,6-9 years old,13
1,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0.0,2016/4,0,0,0,...,0,1,4,run over,none,pedestrian,male,mild injury,40-44 years old,13
2,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0.0,2016/4,0,0,0,...,0,1,4,run over,none,pedestrian,male,mild injury,35-39 years old,13
3,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0.0,2016/4,0,0,0,...,0,1,4,run over,none,pedestrian,female,mild injury,35-39 years old,13
4,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0.0,2016/4,0,0,0,...,0,1,4,run over,car,driver,male,uninjured,30-34 years old,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78031,2018-12-31,21:00-21:59,Monday,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA-,0.0,2019/18,0,0,0,...,0,1,1,multiple crash,car,traveler,male,uninjured,30-34 years old,4
78032,2018-12-31,21:00-21:59,Monday,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA-,0.0,2019/18,0,0,0,...,0,1,1,multiple crash,car,driver,male,uninjured,40-44 years old,4
78033,2018-12-31,23:00-23:59,Monday,HORTALEZA,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,0.0,2019/53,0,0,0,...,0,1,1,other,car,driver,female,uninjured,21-24 years old,16
78034,2018-12-31,23:00-23:59,Monday,HORTALEZA,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,0.0,2019/53,0,0,0,...,0,1,1,other,car,driver,male,mild injury,50-54 years old,16


In [143]:
#clean_accidents.to_excel('./files_for_lab/clean_accidents.xlsx')

In [87]:
unique_accidents = pd.read_sql('''SELECT *,
CASE
WHEN week_day IN ('Monday','Tuesday','Wednesday','Thursday') THEN 'weekday'
ELSE 'weekend'
END AS 'weekday_or_weekend'
FROM mid_camp_project.accidents
GROUP BY id_accident;
''', engine)

In [88]:
unique_accidents

,id,date,hour,week_day,district,street,number,id_accident,number_of_victims,type_of_accident,vehicle,type_of_person,gender,gravity,age,id_district,weekday_or_weekend
0,0,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,4,run over,none,pedestrian,male,mild injury,6-9 years old,13,weekend
1,5,2016-01-01,1:00-1:59,Friday,CIUDAD LINEAL,AVENIDA DEL MARQUES DE CORBERA NUM -,7,2016/437,1,multiple crash,car,driver,female,uninjured,30-34 years old,15,weekend
2,7,2016-01-01,2:00-2:59,Friday,CHAMARTIN,AUTOVIA M-30 CALZADA 1 KM. -,1300,2016/31,1,multiple crash,car,driver,female,mild injury,50-54 years old,5,weekend
3,10,2016-01-01,2:00-2:59,Friday,CHAMARTIN,PASEO DE LA CASTELLANA - PLAZA DE LIMA -,0,2016/255,1,multiple crash,car,driver,female,uninjured,35-39 years old,5,weekend
4,17,2016-01-01,2:00-2:59,Friday,USERA,CALLE DE MARIANO VELA - CALLE DE DOLORES BARRA...,0,2016/3,1,multiple crash,car,driver,male,uninjured,30-34 years old,12,weekend
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31271,78020,2018-12-31,16:00-16:59,Monday,USERA,AVENIDA DE LOS ROSALES NUM -,32,2019/66,1,multiple crash,car,driver,male,uninjured,25-29 years old,12,weekday
31272,78023,2018-12-31,19:00-19:59,Monday,SAN BLAS,CARRETERA DE ACCESO A LA ESTACION DE O'DONNELL...,0,2018/16693,2,multiple crash,car,driver,female,mild injury,35-39 years old,20,weekday
31273,78025,2018-12-31,21:00-21:59,Monday,CENTRO,CALLE DE VILLANUEVA - PASEO DE RECOLETOS -,0,2019/32,1,multiple crash,car,driver,male,uninjured,50-54 years old,1,weekday
31274,78029,2018-12-31,21:00-21:59,Monday,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA -,0,2019/18,1,multiple crash,car,driver,male,mild injury,45-49 years old,4,weekday


In [89]:
#unique_accidents.to_csv('./files_for_lab/unique_accidents.csv')

In [90]:
final_data = clean_accidents.copy()

In [91]:
# I want all data, so i need clean_accidents. I have made a copy and i'm going to delete all rows that aren't in unique_accidents

In [92]:
ids = []
for i in range(len(unique_accidents['id'])):
    ids = ids + [unique_accidents['id'][i]]

In [93]:
final_data = final_data.iloc[ids]

In [94]:
final_data = final_data.reset_index()
final_data

,index,date,hour,week_day,district,street,number,id_accident,ac_hail,ac_ice,...,ice,dry_and_clean,number_of_victims,type_of_accident,vehicle,type_of_person,gender,gravity,age,district_code
0,0,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0.0,2016/4,0,0,...,0,1,4,run over,none,pedestrian,male,mild injury,6-9 years old,13
1,5,2016-01-01,1:00-1:59,Friday,CIUDAD LINEAL,AVENIDA DEL MARQUES DE CORBERA NUM-,7.0,2016/437,0,0,...,0,1,1,multiple crash,car,driver,female,uninjured,30-34 years old,15
2,7,2016-01-01,2:00-2:59,Friday,CHAMARTIN,AUTOVIA M-30 CALZADA 1 KM.-,1300.0,2016/31,0,0,...,0,1,1,multiple crash,car,driver,female,mild injury,50-54 years old,5
3,10,2016-01-01,2:00-2:59,Friday,CHAMARTIN,PASEO DE LA CASTELLANA - PLAZA DE LIMA-,0.0,2016/255,0,0,...,0,1,1,multiple crash,car,driver,female,uninjured,35-39 years old,5
4,17,2016-01-01,2:00-2:59,Friday,USERA,CALLE DE MARIANO VELA - CALLE DE DOLORES BARRA...,0.0,2016/3,0,0,...,0,1,1,multiple crash,car,driver,male,uninjured,30-34 years old,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31271,78020,2018-12-31,16:00-16:59,Monday,USERA,AVENIDA DE LOS ROSALES NUM-,32.0,2019/66,0,0,...,0,1,1,multiple crash,car,driver,male,uninjured,25-29 years old,12
31272,78023,2018-12-31,19:00-19:59,Monday,SAN BLAS,CARRETERA DE ACCESO A LA ESTACION DE O'DONNELL...,0.0,2018/16693,0,0,...,0,1,2,multiple crash,car,driver,female,mild injury,35-39 years old,20
31273,78025,2018-12-31,21:00-21:59,Monday,CENTRO,CALLE DE VILLANUEVA - PASEO DE RECOLETOS-,0.0,2019/32,0,0,...,0,1,1,multiple crash,car,driver,male,uninjured,50-54 years old,1
31274,78029,2018-12-31,21:00-21:59,Monday,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA-,0.0,2019/18,0,0,...,0,1,1,multiple crash,car,driver,male,mild injury,45-49 years old,4


In [95]:
column = unique_accidents['weekday_or_weekend']
final_data = pd.concat([final_data,column], axis=1)
final_data

,index,date,hour,week_day,district,street,number,id_accident,ac_hail,ac_ice,...,dry_and_clean,number_of_victims,type_of_accident,vehicle,type_of_person,gender,gravity,age,district_code,weekday_or_weekend
0,0,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0.0,2016/4,0,0,...,1,4,run over,none,pedestrian,male,mild injury,6-9 years old,13,weekend
1,5,2016-01-01,1:00-1:59,Friday,CIUDAD LINEAL,AVENIDA DEL MARQUES DE CORBERA NUM-,7.0,2016/437,0,0,...,1,1,multiple crash,car,driver,female,uninjured,30-34 years old,15,weekend
2,7,2016-01-01,2:00-2:59,Friday,CHAMARTIN,AUTOVIA M-30 CALZADA 1 KM.-,1300.0,2016/31,0,0,...,1,1,multiple crash,car,driver,female,mild injury,50-54 years old,5,weekend
3,10,2016-01-01,2:00-2:59,Friday,CHAMARTIN,PASEO DE LA CASTELLANA - PLAZA DE LIMA-,0.0,2016/255,0,0,...,1,1,multiple crash,car,driver,female,uninjured,35-39 years old,5,weekend
4,17,2016-01-01,2:00-2:59,Friday,USERA,CALLE DE MARIANO VELA - CALLE DE DOLORES BARRA...,0.0,2016/3,0,0,...,1,1,multiple crash,car,driver,male,uninjured,30-34 years old,12,weekend
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31271,78020,2018-12-31,16:00-16:59,Monday,USERA,AVENIDA DE LOS ROSALES NUM-,32.0,2019/66,0,0,...,1,1,multiple crash,car,driver,male,uninjured,25-29 years old,12,weekday
31272,78023,2018-12-31,19:00-19:59,Monday,SAN BLAS,CARRETERA DE ACCESO A LA ESTACION DE O'DONNELL...,0.0,2018/16693,0,0,...,1,2,multiple crash,car,driver,female,mild injury,35-39 years old,20,weekday
31273,78025,2018-12-31,21:00-21:59,Monday,CENTRO,CALLE DE VILLANUEVA - PASEO DE RECOLETOS-,0.0,2019/32,0,0,...,1,1,multiple crash,car,driver,male,uninjured,50-54 years old,1,weekday
31274,78029,2018-12-31,21:00-21:59,Monday,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA-,0.0,2019/18,0,0,...,1,1,multiple crash,car,driver,male,mild injury,45-49 years old,4,weekday


In [96]:
final_data['weekday_or_weekend'].value_counts(dropna=False)

weekday    19220
weekend    12056
Name: weekday_or_weekend, dtype: int64

In [97]:
unique_accidents['weekday_or_weekend'].value_counts(dropna=False)

weekday    19220
weekend    12056
Name: weekday_or_weekend, dtype: int64

In [98]:
final_data['postal_code'] = 0

In [99]:
final_data.columns

Index(['index', 'date', 'hour', 'week_day', 'district', 'street', 'number',
       'id_accident', 'ac_hail', 'ac_ice', 'ac_rain', 'ac_fog', 'ac_dry',
       'ac_snow', 'wet', 'oil', 'mud', 'gravel', 'ice', 'dry_and_clean',
       'number_of_victims', 'type_of_accident', 'vehicle', 'type_of_person',
       'gender', 'gravity', 'age', 'district_code', 'weekday_or_weekend',
       'postal_code'],
      dtype='object')

In [100]:
ids1 = []
for i in range(len(data['id'])):
    ids1 = ids1 + [data['id'][i]]

In [101]:
data.set_index('id',inplace = True)

In [102]:
data

,date,hour,week_day,district,street,number,id_accident,number_of_victims,type_of_accident,vehicle,type_of_person,gender,gravity,age,id_district,weekday_or_weekend,postal_code
id,,,,,,,,,,,,,,,,,
0,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,4,run over,none,pedestrian,male,mild injury,6-9 years old,13,weekend,28018
1,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,4,run over,none,pedestrian,male,mild injury,40-44 years old,13,weekend,28018
2,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,4,run over,none,pedestrian,male,mild injury,35-39 years old,13,weekend,28018
3,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,4,run over,none,pedestrian,female,mild injury,35-39 years old,13,weekend,28018
4,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,4,run over,car,driver,male,uninjured,30-34 years old,13,weekend,28018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77976,2018-12-30,00:00-00:59,Sunday,CHAMBERI,CALLE DE GUZMAN EL BUENO NUM -,55,2018/16667,2,multiple crash,car,traveler,male,uninjured,18-20 years old,7,weekend,28003
77977,2018-12-30,00:00-00:59,Sunday,CHAMBERI,CALLE DE GUZMAN EL BUENO NUM -,55,2018/16667,2,multiple crash,car,traveler,female,mild injury,50-54 years old,7,weekend,28003
77978,2018-12-30,00:00-00:59,Sunday,CHAMBERI,CALLE DE GUZMAN EL BUENO NUM -,55,2018/16667,2,multiple crash,car,traveler,male,uninjured,unknown,7,weekend,28003


In [103]:
for i in range(len(final_data['postal_code'])):
    if final_data['index'][i] in ids1:
        final_data['postal_code'][i] += data['postal_code'][final_data['index'][i]]

<ipython-input-103-331fa253ae75>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['postal_code'][i] += data['postal_code'][final_data['index'][i]]


In [104]:
final_data['postal_code'].value_counts()

0        7025
28003    1224
28002     853
28016     784
28031     667
28007     649
28041     626
28019     623
28005     608
28028     600
28022     592
28014     580
28011     560
28043     558
28033     544
28021     542
28030     520
28025     510
28017     506
28045     502
28037     488
28054     484
28042     482
28012     473
28035     450
28015     420
28010     419
28027     417
28036     415
28046     409
28040     403
28023     402
28053     397
28020     391
28006     382
28018     381
28026     374
28008     369
28050     365
28039     361
28009     361
28032     356
28034     352
28047     349
28029     305
28013     304
28004     296
28051     291
28038     272
28044     235
28001     228
28024     207
28049     185
28055      85
28052      65
28048      30
Name: postal_code, dtype: int64

In [105]:
data['postal_code'].value_counts()

28003    2889
28002    2148
28016    1800
28031    1759
28041    1712
28007    1616
28019    1596
28022    1517
28014    1497
28028    1495
28021    1437
28005    1426
28043    1391
28054    1363
28025    1357
28011    1343
28017    1332
28033    1330
28030    1282
28045    1223
28037    1221
28042    1133
28035    1092
28012    1082
28027    1077
28046    1071
28040    1037
28036    1032
28053    1006
28018    1001
28015     963
28023     962
28026     960
28010     935
28020     910
28006     900
28008     896
28032     886
28009     879
28047     873
28039     864
28050     840
28034     824
28029     722
28038     720
28051     677
28004     652
28013     652
28044     628
28001     560
28024     540
28049     413
28055     193
28052     146
28048      56
Name: postal_code, dtype: int64

In [106]:
final_data

,index,date,hour,week_day,district,street,number,id_accident,ac_hail,ac_ice,...,number_of_victims,type_of_accident,vehicle,type_of_person,gender,gravity,age,district_code,weekday_or_weekend,postal_code
0,0,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0.0,2016/4,0,0,...,4,run over,none,pedestrian,male,mild injury,6-9 years old,13,weekend,28018
1,5,2016-01-01,1:00-1:59,Friday,CIUDAD LINEAL,AVENIDA DEL MARQUES DE CORBERA NUM-,7.0,2016/437,0,0,...,1,multiple crash,car,driver,female,uninjured,30-34 years old,15,weekend,28017
2,7,2016-01-01,2:00-2:59,Friday,CHAMARTIN,AUTOVIA M-30 CALZADA 1 KM.-,1300.0,2016/31,0,0,...,1,multiple crash,car,driver,female,mild injury,50-54 years old,5,weekend,0
3,10,2016-01-01,2:00-2:59,Friday,CHAMARTIN,PASEO DE LA CASTELLANA - PLAZA DE LIMA-,0.0,2016/255,0,0,...,1,multiple crash,car,driver,female,uninjured,35-39 years old,5,weekend,28046
4,17,2016-01-01,2:00-2:59,Friday,USERA,CALLE DE MARIANO VELA - CALLE DE DOLORES BARRA...,0.0,2016/3,0,0,...,1,multiple crash,car,driver,male,uninjured,30-34 years old,12,weekend,28026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31271,78020,2018-12-31,16:00-16:59,Monday,USERA,AVENIDA DE LOS ROSALES NUM-,32.0,2019/66,0,0,...,1,multiple crash,car,driver,male,uninjured,25-29 years old,12,weekday,0
31272,78023,2018-12-31,19:00-19:59,Monday,SAN BLAS,CARRETERA DE ACCESO A LA ESTACION DE O'DONNELL...,0.0,2018/16693,0,0,...,2,multiple crash,car,driver,female,mild injury,35-39 years old,20,weekday,28022
31273,78025,2018-12-31,21:00-21:59,Monday,CENTRO,CALLE DE VILLANUEVA - PASEO DE RECOLETOS-,0.0,2019/32,0,0,...,1,multiple crash,car,driver,male,uninjured,50-54 years old,1,weekday,28005
31274,78029,2018-12-31,21:00-21:59,Monday,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA-,0.0,2019/18,0,0,...,1,multiple crash,car,driver,male,mild injury,45-49 years old,4,weekday,28001


In [107]:
#final_data.to_excel('./files_for_lab/final_data.xlsx')

In [108]:
useless = final_data[final_data['postal_code'] == 0]
useless

,index,date,hour,week_day,district,street,number,id_accident,ac_hail,ac_ice,...,number_of_victims,type_of_accident,vehicle,type_of_person,gender,gravity,age,district_code,weekday_or_weekend,postal_code
2,7,2016-01-01,2:00-2:59,Friday,CHAMARTIN,AUTOVIA M-30 CALZADA 1 KM.-,1300.0,2016/31,0,0,...,1,multiple crash,car,driver,female,mild injury,50-54 years old,5,weekend,0
13,36,2016-01-01,18:00-18:59,Friday,PUENTE DE VALLECAS,AVENIDA DEL MEDITERRANEO KM.-,3200.0,2016/22,0,0,...,2,multiple crash,car,driver,male,mild injury,21-24 years old,13,weekend,0
18,54,2016-01-01,20:00-20:59,Friday,PUENTE DE VALLECAS,AUTOVIA M-30 CALZADA 2 KM.-,10300.0,2016/86,0,0,...,1,crash with a fixed object,car,driver,male,mild injury,18-20 years old,13,weekend,0
24,64,2016-01-01,23:00-23:59,Friday,CHAMARTIN,CALLE DEL PRINCIPE DE VERGARA NUM-,268.0,2016/63,0,0,...,1,other,other,driver,male,mild injury,30-34 years old,5,weekend,0
26,68,2016-01-02,1:00-1:59,Saturday,FUENCARRAL-EL PARDO,AUTOVIA M-30 CALZADA 2 KM.-,30400.0,2016/25,0,0,...,1,crash with a fixed object,car,driver,male,uninjured,30-34 years old,8,weekend,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31261,77996,2018-12-30,20:00-20:59,Sunday,PUENTE DE VALLECAS,ACCESO A LA AVENIDA DEL MEDITERRANEO-,0.0,2018/16659,0,0,...,1,crash with a fixed object,car,driver,male,uninjured,30-34 years old,13,weekend,0
31264,78003,2018-12-31,13:00-13:59,Monday,MORATALAZ,CALLE DEL CAMINO DE LOS VINATEROS NUM-,117.0,2019/72,0,0,...,1,multiple crash,car,driver,male,uninjured,40-44 years old,14,weekday,0
31265,78005,2018-12-31,13:00-13:59,Monday,TETUAN,CALLE DE LA CASTELLANA - PLAZA DE LIMA-,0.0,2019/38,0,0,...,1,multiple crash,car,driver,female,uninjured,21-24 years old,6,weekday,0
31268,78011,2018-12-31,14:00-14:59,Monday,USERA,PASEO DE LA ERMITA DEL SANTO - PASEO DEL QUINC...,0.0,2019/14,0,0,...,2,crash with a fixed object,car,driver,female,mild injury,21-24 years old,12,weekday,0


In [109]:
useless['street'].value_counts()

AUTOVIA  M-30 CALZADA 1 KM.-                 811
AUTOVIA  M-30 CALZADA 2 KM.-                 625
CALLE DE ALCALA NUM-                         458
PASEO DE LA CASTELLANA NUM-                  357
AVENIDA DEL MEDITERRANEO KM.-                143
                                            ... 
CALLE  30  18RT KM.-                           1
ACCESO  RADIAL 3 - AUTOVIA  M-45-              1
CALLE DE ALCOCER - PASEO DE LOS TALLERES-      1
POBLADO DE SANTA CATALINA-                     1
CALLE DE JAEN - CALLE DE SAN ANTONIO-          1
Name: street, Length: 876, dtype: int64

In [110]:
definitive_data = pd.read_sql('''SELECT ua.*, pc.postal_code
FROM mid_camp_project.unique_accidents ua
INNER JOIN mid_camp_project.code_postal_code cpc USING(id_district)
INNER JOIN mid_camp_project.postal_code pc USING(id_street)
WHERE ((ua.street LIKE pc.type_of_street) AND (ua.street LIKE pc.street_name) AND (ua.number BETWEEN pc.start_of_the_street AND pc.end_of_the_street))
OR ((ua.street LIKE pc.type_of_street) AND (ua.street LIKE pc.street_name) AND (ua.number BETWEEN pc.start_of_the_street AND pc.end_of_the_street))
OR ((ua.street LIKE pc.street_name) AND (ua.number BETWEEN pc.start_of_the_street AND pc.end_of_the_street))
OR ((ua.street LIKE pc.type_of_street) AND (ua.street LIKE pc.street_name)) 
OR ua.street LIKE pc.street_name 
GROUP BY ua.id_accident
ORDER by ua.id ASC;
''', engine)

In [111]:
usefull_data = pd.read_sql('''SELECT ua.*
FROM mid_camp_project.unique_accidents ua
WHERE NOT EXISTS (
	SELECT * FROM mid_camp_project.final f
	WHERE ua.id = f.id 
    );
''', engine)

In [112]:
# I'm going to choose one random postal_code from each district to this usefull data 

In [113]:
codes

,DISTRICT,CODE
0,USERA,12
1,SAN BLAS,20
2,FUENCARRAL-EL PARDO,8
3,ARGANZUELA,2
4,SALAMANCA,4
5,CARABANCHEL,11
6,TETUAN,6
7,CHAMBERI,7
8,VILLAVERDE,17
9,PUENTE DE VALLECAS,13


In [114]:
district_1 = [] # Here i'm saving all the unique postal_codes to use it into the random
district_2 = []
district_3 = []
district_4 = []
district_5 = []
district_6 = []
district_7 = []
district_8 = []
district_9 = []
district_10 = []
district_11 = []
district_12 = []
district_13 = []
district_14 = []
district_15 = []
district_16 = []
district_17 = []
district_18 = []
district_19 = []
district_20 = []
district_21 = []

In [115]:
definitive_data

,id,second_id,date,hour,week_day,district,street,number,id_accident,number_of_victims,type_of_accident,vehicle,type_of_person,gender,gravity,age,id_district,weekday_or_weekend,postal_code
0,0,0,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,4,run over,none,pedestrian,male,mild injury,6-9 years old,13,weekend\r,28018
1,1,5,2016-01-01,1:00-1:59,Friday,CIUDAD LINEAL,AVENIDA DEL MARQUES DE CORBERA NUM,7,2016/437,1,multiple crash,car,driver,female,uninjured,30-34 years old,15,weekend\r,28017
2,3,10,2016-01-01,2:00-2:59,Friday,CHAMARTIN,PASEO DE LA CASTELLANA - PLAZA DE LIMA,0,2016/255,1,multiple crash,car,driver,female,uninjured,35-39 years old,5,weekend\r,28046
3,4,17,2016-01-01,2:00-2:59,Friday,USERA,CALLE DE MARIANO VELA - CALLE DE DOLORES BARRANCO,0,2016/3,1,multiple crash,car,driver,male,uninjured,30-34 years old,12,weekend\r,28026
4,5,22,2016-01-01,3:00-3:59,Friday,LATINA,CALLE DE OCAÑA - CALLE DE VALMOJADO,0,2016/6,1,run over,none,pedestrian,male,serious injury,15-17 years old,10,weekend\r,28047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24246,31270,78016,2018-12-31,16:00-16:59,Monday,PUENTE DE VALLECAS,CALLE DE ROMEO Y JULIETA - PARQUE DE PALOMERAS...,0,2019/95,1,multiple crash,car,driver,male,uninjured,18-20 years old,13,weekday\r,28018
24247,31272,78023,2018-12-31,19:00-19:59,Monday,SAN BLAS,CARRETERA DE ACCESO A LA ESTACION DE O'DONNELL...,0,2018/16693,2,multiple crash,car,driver,female,mild injury,35-39 years old,20,weekday\r,28022
24248,31273,78025,2018-12-31,21:00-21:59,Monday,CENTRO,CALLE DE VILLANUEVA - PASEO DE RECOLETOS,0,2019/32,1,multiple crash,car,driver,male,uninjured,50-54 years old,1,weekday\r,28005
24249,31274,78029,2018-12-31,21:00-21:59,Monday,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA,0,2019/18,1,multiple crash,car,driver,male,mild injury,45-49 years old,4,weekday\r,28001


In [116]:
for i in range(len(definitive_data['postal_code'])):
    if (definitive_data['district'][i] == 'CENTRO') & (definitive_data['postal_code'][i] not in district_1):
        district_1 = district_1 + [definitive_data['postal_code'][i]]
    if (definitive_data['district'][i] == 'ARGANZUELA') & (definitive_data['postal_code'][i] not in district_2):
        district_2 = district_2 + [definitive_data['postal_code'][i]]
    if (definitive_data['district'][i] == 'RETIRO') & (definitive_data['postal_code'][i] not in district_3):
        district_3 = district_3 + [definitive_data['postal_code'][i]]
    if (definitive_data['district'][i] == 'SALAMANCA') & (definitive_data['postal_code'][i] not in district_4):
        district_4 = district_4 + [definitive_data['postal_code'][i]]
    if (definitive_data['district'][i] == 'CHAMARTIN') & (definitive_data['postal_code'][i] not in district_5):
        district_5 = district_5 + [definitive_data['postal_code'][i]]
    if (definitive_data['district'][i] == 'TETUAN') & (definitive_data['postal_code'][i] not in district_6):
        district_6 = district_6 + [definitive_data['postal_code'][i]]
    if (definitive_data['district'][i] == 'CHAMBERI') & (definitive_data['postal_code'][i] not in district_7):
        district_7 = district_7 + [definitive_data['postal_code'][i]]
    if (definitive_data['district'][i] == 'FUENCARRAL-EL PARDO') & (definitive_data['postal_code'][i] not in district_8):
        district_8 = district_8 + [definitive_data['postal_code'][i]]
    if (definitive_data['district'][i] == 'MONCLOA-ARAVACA') & (definitive_data['postal_code'][i] not in district_9):
        district_9 = district_9 + [definitive_data['postal_code'][i]]
    if (definitive_data['district'][i] == 'LATINA') & (definitive_data['postal_code'][i] not in district_10):
        district_10 = district_10 + [definitive_data['postal_code'][i]]
    if (definitive_data['district'][i] == 'CARABANCHEL') & (definitive_data['postal_code'][i] not in district_11):
        district_11 = district_11 + [definitive_data['postal_code'][i]]
    if (definitive_data['district'][i] == 'USERA') & (definitive_data['postal_code'][i] not in district_12):
        district_12 = district_12 + [definitive_data['postal_code'][i]]
    if (definitive_data['district'][i] == 'PUENTE DE VALLECAS') & (definitive_data['postal_code'][i] not in district_13):
        district_13 = district_13 + [definitive_data['postal_code'][i]]
    if (definitive_data['district'][i] == 'MORATALAZ') & (definitive_data['postal_code'][i] not in district_14):
        district_14 = district_14 + [definitive_data['postal_code'][i]]
    if (definitive_data['district'][i] == 'CIUDAD LINEAL') & (definitive_data['postal_code'][i] not in district_15):
        district_15 = district_15 + [definitive_data['postal_code'][i]]
    if (definitive_data['district'][i] == 'HORTALEZA') & (definitive_data['postal_code'][i] not in district_16):
        district_16 = district_16 + [definitive_data['postal_code'][i]]
    if (definitive_data['district'][i] == 'VILLAVERDE') & (definitive_data['postal_code'][i] not in district_17):
        district_17 = district_17 + [definitive_data['postal_code'][i]]
    if (definitive_data['district'][i] == 'VILLA DE VALLECAS') & (definitive_data['postal_code'][i] not in district_18):
        district_18 = district_18 + [definitive_data['postal_code'][i]]
    if (definitive_data['district'][i] == 'VICALVARO') & (definitive_data['postal_code'][i] not in district_19):
        district_19 = district_19 + [definitive_data['postal_code'][i]]
    if (definitive_data['district'][i] == 'SAN BLAS') & (definitive_data['postal_code'][i] not in district_20):
        district_20 = district_20 + [definitive_data['postal_code'][i]]
    if (definitive_data['district'][i] == 'BARAJAS') & (definitive_data['postal_code'][i] not in district_21):
        district_21 = district_21 + [definitive_data['postal_code'][i]]

In [117]:
display(district_1)
display(district_2)
display(district_3)
display(district_4)
display(district_5)
display(district_6)
display(district_7) 
display(district_8)
display(district_9)
display(district_10)
display(district_11) 
display(district_12) 
display(district_13)  
display(district_14)  
display(district_15)  
display(district_16)  
display(district_17)  
display(district_18)  
display(district_19) 
display(district_20)
display(district_21) 

[28012, 28013, 28005, 28004, 28001, 28011, 28014, 28008, 28010, 28015]

[28045, 28005, 28019, 28007, 28012]

[28007, 28014, 28009, 28022, 28001]

[28002, 28009, 28028, 28001, 28006, 28016, 28046]

[28046, 28016, 28036, 28002, 28006, 28033, 28050]

...

[28021, 28041, 28051]

[28051, 28031, 28053]

[28032, 28031, 28052, 28022]

[28037, 28032, 28022, 28027]

[28042, 28055, 28022]

In [118]:
usefull_data['postal_code'] = 0

In [119]:
for i in range(len(usefull_data['postal_code'])):
    if (usefull_data['district'][i] == 'CENTRO'):
        usefull_data['postal_code'][i] = random.choice(district_1)
    if (usefull_data['district'][i] == 'ARGANZUELA'):
        usefull_data['postal_code'][i] = random.choice(district_2)
    if (usefull_data['district'][i] == 'RETIRO'):
        usefull_data['postal_code'][i] = random.choice(district_3)
    if (usefull_data['district'][i] == 'SALAMANCA'):
        usefull_data['postal_code'][i] = random.choice(district_4)
    if (usefull_data['district'][i] == 'CHAMARTIN'):
        usefull_data['postal_code'][i] = random.choice(district_5)
    if (usefull_data['district'][i] == 'TETUAN'):
        usefull_data['postal_code'][i] = random.choice(district_6)
    if (usefull_data['district'][i] == 'CHAMBERI'):
        usefull_data['postal_code'][i] = random.choice(district_7)
    if (usefull_data['district'][i] == 'FUENCARRAL-EL PARDO'):
        usefull_data['postal_code'][i] = random.choice(district_8)
    if (usefull_data['district'][i] == 'MONCLOA-ARAVACA'):
        usefull_data['postal_code'][i] = random.choice(district_9)
    if (usefull_data['district'][i] == 'LATINA'):
        usefull_data['postal_code'][i] = random.choice(district_10)
    if (usefull_data['district'][i] == 'CARABANCHEL'):
        usefull_data['postal_code'][i] = random.choice(district_11)
    if (usefull_data['district'][i] == 'USERA'):
        usefull_data['postal_code'][i] = random.choice(district_12)
    if (usefull_data['district'][i] == 'PUENTE DE VALLECAS'):
        usefull_data['postal_code'][i] = random.choice(district_13)
    if (usefull_data['district'][i] == 'MORATALAZ'):
        usefull_data['postal_code'][i] = random.choice(district_14)
    if (usefull_data['district'][i] == 'CIUDAD LINEAL'):
        usefull_data['postal_code'][i] = random.choice(district_15)
    if (usefull_data['district'][i] == 'HORTALEZA'):
        usefull_data['postal_code'][i] = random.choice(district_16)
    if (usefull_data['district'][i] == 'VILLAVERDE'):
        usefull_data['postal_code'][i] = random.choice(district_17)
    if (usefull_data['district'][i] == 'VILLA DE VALLECAS'):
        usefull_data['postal_code'][i] = random.choice(district_18)
    if (usefull_data['district'][i] == 'VICALVARO'):
        usefull_data['postal_code'][i] = random.choice(district_19)
    if (usefull_data['district'][i] == 'SAN BLAS'):
        usefull_data['postal_code'][i] = random.choice(district_20)
    if (usefull_data['district'][i] == 'BARAJAS'):
        usefull_data['postal_code'][i] = random.choice(district_21)

<ipython-input-119-e8fa60bd7b1e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usefull_data['postal_code'][i] = random.choice(district_5)
<ipython-input-119-e8fa60bd7b1e>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usefull_data['postal_code'][i] = random.choice(district_13)
<ipython-input-119-e8fa60bd7b1e>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usefull_data['postal_code'][i] = random.choice(district_8)
<ipython-inp

In [120]:
usefull_data['postal_code'].value_counts()

28007    385
28022    292
28040    236
28001    235
28053    232
28009    208
28041    208
28030    204
28031    201
28033    199
28050    192
28032    186
28003    183
28012    169
28019    161
28028    154
28046    153
28006    149
28027    140
28045    135
28016    135
28029    134
28002    130
28038    129
28026    127
28043    127
28039    124
28018    123
28005    122
28011    113
28010    110
28047    109
28014    107
28015    106
28025    105
28037     95
28035     93
28054     90
28044     89
28020     85
28051     81
28017     80
28055     75
28042     74
28048     60
28036     60
28034     58
28049     55
28024     52
28008     50
28023     44
28021     25
28052     22
28013      7
28004      7
Name: postal_code, dtype: int64

In [121]:
usefull_data

,id,second_id,date,hour,week_day,district,street,number,id_accident,number_of_victims,type_of_accident,vehicle,type_of_person,gender,gravity,age,id_district,weekday_or_weekend,postal_code
0,2,7,2016-01-01,2:00-2:59,Friday,CHAMARTIN,AUTOVIA M-30 CALZADA 1 KM.,1300,2016/31,1,multiple crash,car,driver,female,mild injury,50-54 years old,5,weekend\r,28036
1,13,36,2016-01-01,18:00-18:59,Friday,PUENTE DE VALLECAS,AVENIDA DEL MEDITERRANEO KM.,3200,2016/22,2,multiple crash,car,driver,male,mild injury,21-24 years old,13,weekend\r,28053
2,18,54,2016-01-01,20:00-20:59,Friday,PUENTE DE VALLECAS,AUTOVIA M-30 CALZADA 2 KM.,10300,2016/86,1,crash with a fixed object,car,driver,male,mild injury,18-20 years old,13,weekend\r,28038
3,24,64,2016-01-01,23:00-23:59,Friday,CHAMARTIN,CALLE DEL PRINCIPE DE VERGARA NUM,268,2016/63,1,other,other,driver,male,mild injury,30-34 years old,5,weekend\r,28050
4,26,68,2016-01-02,1:00-1:59,Saturday,FUENCARRAL-EL PARDO,AUTOVIA M-30 CALZADA 2 KM.,30400,2016/25,1,crash with a fixed object,car,driver,male,uninjured,30-34 years old,8,weekend\r,28034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7020,31261,77996,2018-12-30,20:00-20:59,Sunday,PUENTE DE VALLECAS,ACCESO A LA AVENIDA DEL MEDITERRANEO,0,2018/16659,1,crash with a fixed object,car,driver,male,uninjured,30-34 years old,13,weekend\r,28018
7021,31264,78003,2018-12-31,13:00-13:59,Monday,MORATALAZ,CALLE DEL CAMINO DE LOS VINATEROS NUM,117,2019/72,1,multiple crash,car,driver,male,uninjured,40-44 years old,14,weekday\r,28030
7022,31265,78005,2018-12-31,13:00-13:59,Monday,TETUAN,CALLE DE LA CASTELLANA - PLAZA DE LIMA,0,2019/38,1,multiple crash,car,driver,female,uninjured,21-24 years old,6,weekday\r,28003
7023,31268,78011,2018-12-31,14:00-14:59,Monday,USERA,PASEO DE LA ERMITA DEL SANTO - PASEO DEL QUINC...,0,2019/14,2,crash with a fixed object,car,driver,female,mild injury,21-24 years old,12,weekday\r,28012


In [122]:
complete_data = pd.concat([definitive_data, usefull_data], axis = 0)

In [123]:
complete_data = complete_data.sort_values('id')

In [124]:
complete_data

,id,second_id,date,hour,week_day,district,street,number,id_accident,number_of_victims,type_of_accident,vehicle,type_of_person,gender,gravity,age,id_district,weekday_or_weekend,postal_code
0,0,0,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,4,run over,none,pedestrian,male,mild injury,6-9 years old,13,weekend\r,28018
1,1,5,2016-01-01,1:00-1:59,Friday,CIUDAD LINEAL,AVENIDA DEL MARQUES DE CORBERA NUM,7,2016/437,1,multiple crash,car,driver,female,uninjured,30-34 years old,15,weekend\r,28017
0,2,7,2016-01-01,2:00-2:59,Friday,CHAMARTIN,AUTOVIA M-30 CALZADA 1 KM.,1300,2016/31,1,multiple crash,car,driver,female,mild injury,50-54 years old,5,weekend\r,28036
2,3,10,2016-01-01,2:00-2:59,Friday,CHAMARTIN,PASEO DE LA CASTELLANA - PLAZA DE LIMA,0,2016/255,1,multiple crash,car,driver,female,uninjured,35-39 years old,5,weekend\r,28046
3,4,17,2016-01-01,2:00-2:59,Friday,USERA,CALLE DE MARIANO VELA - CALLE DE DOLORES BARRANCO,0,2016/3,1,multiple crash,car,driver,male,uninjured,30-34 years old,12,weekend\r,28026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7024,31271,78020,2018-12-31,16:00-16:59,Monday,USERA,AVENIDA DE LOS ROSALES NUM,32,2019/66,1,multiple crash,car,driver,male,uninjured,25-29 years old,12,weekday\r,28041
24247,31272,78023,2018-12-31,19:00-19:59,Monday,SAN BLAS,CARRETERA DE ACCESO A LA ESTACION DE O'DONNELL...,0,2018/16693,2,multiple crash,car,driver,female,mild injury,35-39 years old,20,weekday\r,28022
24248,31273,78025,2018-12-31,21:00-21:59,Monday,CENTRO,CALLE DE VILLANUEVA - PASEO DE RECOLETOS,0,2019/32,1,multiple crash,car,driver,male,uninjured,50-54 years old,1,weekday\r,28005
24249,31274,78029,2018-12-31,21:00-21:59,Monday,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA,0,2019/18,1,multiple crash,car,driver,male,mild injury,45-49 years old,4,weekday\r,28001


In [125]:
 complete_data['weekday_or_weekend'] =  complete_data['weekday_or_weekend'].replace({'\r':''}, regex=True)

In [126]:
 complete_data['weekday_or_weekend']

0        weekend
1        weekend
0        weekend
2        weekend
3        weekend
          ...   
7024     weekday
24247    weekday
24248    weekday
24249    weekday
24250    weekday
Name: weekday_or_weekend, Length: 31276, dtype: object

In [127]:
complete_data = complete_data.reset_index(drop=True)

In [128]:
complete_data

,id,second_id,date,hour,week_day,district,street,number,id_accident,number_of_victims,type_of_accident,vehicle,type_of_person,gender,gravity,age,id_district,weekday_or_weekend,postal_code
0,0,0,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0,2016/4,4,run over,none,pedestrian,male,mild injury,6-9 years old,13,weekend,28018
1,1,5,2016-01-01,1:00-1:59,Friday,CIUDAD LINEAL,AVENIDA DEL MARQUES DE CORBERA NUM,7,2016/437,1,multiple crash,car,driver,female,uninjured,30-34 years old,15,weekend,28017
2,2,7,2016-01-01,2:00-2:59,Friday,CHAMARTIN,AUTOVIA M-30 CALZADA 1 KM.,1300,2016/31,1,multiple crash,car,driver,female,mild injury,50-54 years old,5,weekend,28036
3,3,10,2016-01-01,2:00-2:59,Friday,CHAMARTIN,PASEO DE LA CASTELLANA - PLAZA DE LIMA,0,2016/255,1,multiple crash,car,driver,female,uninjured,35-39 years old,5,weekend,28046
4,4,17,2016-01-01,2:00-2:59,Friday,USERA,CALLE DE MARIANO VELA - CALLE DE DOLORES BARRANCO,0,2016/3,1,multiple crash,car,driver,male,uninjured,30-34 years old,12,weekend,28026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31271,31271,78020,2018-12-31,16:00-16:59,Monday,USERA,AVENIDA DE LOS ROSALES NUM,32,2019/66,1,multiple crash,car,driver,male,uninjured,25-29 years old,12,weekday,28041
31272,31272,78023,2018-12-31,19:00-19:59,Monday,SAN BLAS,CARRETERA DE ACCESO A LA ESTACION DE O'DONNELL...,0,2018/16693,2,multiple crash,car,driver,female,mild injury,35-39 years old,20,weekday,28022
31273,31273,78025,2018-12-31,21:00-21:59,Monday,CENTRO,CALLE DE VILLANUEVA - PASEO DE RECOLETOS,0,2019/32,1,multiple crash,car,driver,male,uninjured,50-54 years old,1,weekday,28005
31274,31274,78029,2018-12-31,21:00-21:59,Monday,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA,0,2019/18,1,multiple crash,car,driver,male,mild injury,45-49 years old,4,weekday,28001


In [129]:
ids2 = []
for i in range(len(complete_data['second_id'])):
    ids2 = ids2 + [complete_data['second_id'][i]]

In [130]:
clean_accidents

,date,hour,week_day,district,street,number,id_accident,ac_hail,ac_ice,ac_rain,...,ice,dry_and_clean,number_of_victims,type_of_accident,vehicle,type_of_person,gender,gravity,age,district_code
0,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0.0,2016/4,0,0,0,...,0,1,4,run over,none,pedestrian,male,mild injury,6-9 years old,13
1,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0.0,2016/4,0,0,0,...,0,1,4,run over,none,pedestrian,male,mild injury,40-44 years old,13
2,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0.0,2016/4,0,0,0,...,0,1,4,run over,none,pedestrian,male,mild injury,35-39 years old,13
3,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0.0,2016/4,0,0,0,...,0,1,4,run over,none,pedestrian,female,mild injury,35-39 years old,13
4,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0.0,2016/4,0,0,0,...,0,1,4,run over,car,driver,male,uninjured,30-34 years old,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78031,2018-12-31,21:00-21:59,Monday,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA-,0.0,2019/18,0,0,0,...,0,1,1,multiple crash,car,traveler,male,uninjured,30-34 years old,4
78032,2018-12-31,21:00-21:59,Monday,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA-,0.0,2019/18,0,0,0,...,0,1,1,multiple crash,car,driver,male,uninjured,40-44 years old,4
78033,2018-12-31,23:00-23:59,Monday,HORTALEZA,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,0.0,2019/53,0,0,0,...,0,1,1,other,car,driver,female,uninjured,21-24 years old,16
78034,2018-12-31,23:00-23:59,Monday,HORTALEZA,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,0.0,2019/53,0,0,0,...,0,1,1,other,car,driver,male,mild injury,50-54 years old,16


In [131]:
definitive_dataframe = clean_accidents.copy()

In [132]:
definitive_dataframe = definitive_dataframe.loc[ids2]

In [133]:
definitive_dataframe

,date,hour,week_day,district,street,number,id_accident,ac_hail,ac_ice,ac_rain,...,ice,dry_and_clean,number_of_victims,type_of_accident,vehicle,type_of_person,gender,gravity,age,district_code
0,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0.0,2016/4,0,0,0,...,0,1,4,run over,none,pedestrian,male,mild injury,6-9 years old,13
5,2016-01-01,1:00-1:59,Friday,CIUDAD LINEAL,AVENIDA DEL MARQUES DE CORBERA NUM-,7.0,2016/437,0,0,0,...,0,1,1,multiple crash,car,driver,female,uninjured,30-34 years old,15
7,2016-01-01,2:00-2:59,Friday,CHAMARTIN,AUTOVIA M-30 CALZADA 1 KM.-,1300.0,2016/31,0,0,0,...,0,1,1,multiple crash,car,driver,female,mild injury,50-54 years old,5
10,2016-01-01,2:00-2:59,Friday,CHAMARTIN,PASEO DE LA CASTELLANA - PLAZA DE LIMA-,0.0,2016/255,0,0,0,...,0,1,1,multiple crash,car,driver,female,uninjured,35-39 years old,5
17,2016-01-01,2:00-2:59,Friday,USERA,CALLE DE MARIANO VELA - CALLE DE DOLORES BARRA...,0.0,2016/3,0,0,0,...,0,1,1,multiple crash,car,driver,male,uninjured,30-34 years old,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78020,2018-12-31,16:00-16:59,Monday,USERA,AVENIDA DE LOS ROSALES NUM-,32.0,2019/66,0,0,0,...,0,1,1,multiple crash,car,driver,male,uninjured,25-29 years old,12
78023,2018-12-31,19:00-19:59,Monday,SAN BLAS,CARRETERA DE ACCESO A LA ESTACION DE O'DONNELL...,0.0,2018/16693,0,0,0,...,0,1,2,multiple crash,car,driver,female,mild injury,35-39 years old,20
78025,2018-12-31,21:00-21:59,Monday,CENTRO,CALLE DE VILLANUEVA - PASEO DE RECOLETOS-,0.0,2019/32,0,0,0,...,0,1,1,multiple crash,car,driver,male,uninjured,50-54 years old,1
78029,2018-12-31,21:00-21:59,Monday,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA-,0.0,2019/18,0,0,0,...,0,1,1,multiple crash,car,driver,male,mild injury,45-49 years old,4


In [134]:
definitive_dataframe = definitive_dataframe.reset_index(drop=True)

In [135]:
final_columns = complete_data[['weekday_or_weekend', 'postal_code']]
final_columns

,weekday_or_weekend,postal_code
0,weekend,28018
1,weekend,28017
2,weekend,28036
3,weekend,28046
4,weekend,28026
...,...,...
31271,weekday,28041
31272,weekday,28022
31273,weekday,28005
31274,weekday,28001


In [136]:
data_to_tableau = pd.concat([definitive_dataframe, final_columns], axis= 1)
data_to_tableau

,date,hour,week_day,district,street,number,id_accident,ac_hail,ac_ice,ac_rain,...,number_of_victims,type_of_accident,vehicle,type_of_person,gender,gravity,age,district_code,weekday_or_weekend,postal_code
0,2016-01-01,00:00-00:59,Friday,PUENTE DE VALLECAS,AVENIDA DE PABLO NERUDA - CALLE DEL GOLFO DE D...,0.0,2016/4,0,0,0,...,4,run over,none,pedestrian,male,mild injury,6-9 years old,13,weekend,28018
1,2016-01-01,1:00-1:59,Friday,CIUDAD LINEAL,AVENIDA DEL MARQUES DE CORBERA NUM-,7.0,2016/437,0,0,0,...,1,multiple crash,car,driver,female,uninjured,30-34 years old,15,weekend,28017
2,2016-01-01,2:00-2:59,Friday,CHAMARTIN,AUTOVIA M-30 CALZADA 1 KM.-,1300.0,2016/31,0,0,0,...,1,multiple crash,car,driver,female,mild injury,50-54 years old,5,weekend,28036
3,2016-01-01,2:00-2:59,Friday,CHAMARTIN,PASEO DE LA CASTELLANA - PLAZA DE LIMA-,0.0,2016/255,0,0,0,...,1,multiple crash,car,driver,female,uninjured,35-39 years old,5,weekend,28046
4,2016-01-01,2:00-2:59,Friday,USERA,CALLE DE MARIANO VELA - CALLE DE DOLORES BARRA...,0.0,2016/3,0,0,0,...,1,multiple crash,car,driver,male,uninjured,30-34 years old,12,weekend,28026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31271,2018-12-31,16:00-16:59,Monday,USERA,AVENIDA DE LOS ROSALES NUM-,32.0,2019/66,0,0,0,...,1,multiple crash,car,driver,male,uninjured,25-29 years old,12,weekday,28041
31272,2018-12-31,19:00-19:59,Monday,SAN BLAS,CARRETERA DE ACCESO A LA ESTACION DE O'DONNELL...,0.0,2018/16693,0,0,0,...,2,multiple crash,car,driver,female,mild injury,35-39 years old,20,weekday,28022
31273,2018-12-31,21:00-21:59,Monday,CENTRO,CALLE DE VILLANUEVA - PASEO DE RECOLETOS-,0.0,2019/32,0,0,0,...,1,multiple crash,car,driver,male,uninjured,50-54 years old,1,weekday,28005
31274,2018-12-31,21:00-21:59,Monday,SALAMANCA,CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA-,0.0,2019/18,0,0,0,...,1,multiple crash,car,driver,male,mild injury,45-49 years old,4,weekday,28001


In [137]:
#data_to_tableau.to_excel('./files_for_lab/data_to_tableau.xlsx')

In [138]:
data_to_tableau['postal_code'].value_counts()

28003    1081
28007    1035
28002    1005
28016     927
28022     915
28031     875
28041     838
28019     787
28028     742
28020     741
28043     730
28030     724
28005     705
28014     702
28011     678
28033     668
28045     644
28040     642
28053     635
28012     628
28025     605
28017     586
28027     585
28054     571
28021     568
28039     568
28035     566
28046     562
28050     557
28042     556
28037     552
28009     545
28032     539
28010     538
28006     535
28015     517
28018     497
28026     487
28001     475
28036     475
28047     462
28023     454
28008     422
28029     416
28038     399
28034     380
28051     372
28044     341
28013     312
28004     290
28024     258
28049     243
28055     162
28048      92
28052      87
Name: postal_code, dtype: int64

In [139]:
postal_codes_tableau = data_to_tableau['postal_code'].unique().tolist()
postal_codes_tableau

[28018,
 28017,
 28036,
 28046,
 28026,
 28047,
 28007,
 28031,
 28021,
 28053,
 28002,
 28010,
 28045,
 28016,
 28020,
 28005,
 28038,
 28009,
 28035,
 28033,
 28050,
 28034,
 28012,
 28028,
 28042,
 28025,
 28019,
 28051,
 28039,
 28013,
 28049,
 28027,
 28054,
 28048,
 28032,
 28006,
 28001,
 28037,
 28004,
 28011,
 28044,
 28022,
 28043,
 28029,
 28014,
 28023,
 28030,
 28003,
 28040,
 28055,
 28041,
 28008,
 28024,
 28015,
 28052]

In [140]:
r = 0
for code in postal_codes_tableau:
    a = data_to_tableau[data_to_tableau['postal_code'] == code]
    liist = a['district'].unique().tolist()
    if len(liist) > 1:
        display(code)
        display(a['district'].value_counts())
        r += 1

28046

CHAMARTIN    451
SALAMANCA    111
Name: district, dtype: int64

28026

USERA     425
LATINA     62
Name: district, dtype: int64

28047

...

USERA          401
VILLAVERDE     228
LATINA         126
CARABANCHEL     83
Name: district, dtype: int64

28008

MONCLOA-ARAVACA    325
CENTRO              97
Name: district, dtype: int64

28015

CHAMBERI    259
CENTRO      258
Name: district, dtype: int64

In [141]:
r

39